 #  ▌Ufo and_Beyond(2010-2020) - ETL Process and GEO Coding ▌

In [1]:
# Dependencies
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime as dt
import seaborn as sns
#from plotnine import *
#import maya
import dateutil.parser
import sys
import pandas as pd
from pprint import pprint
from opencage.geocoder import OpenCageGeocode
from opencage.geocoder import InvalidInputError, RateLimitExceededError, UnknownError

# Import API key
from api_keys import key

## DATA Sourcing
 - For the project we sourced data from data world by Timothy Rener (https://data.world/timothyrenner/ufo-sightings).
 
 - The dataset was assembled by scraping the reports from the NUFORC site. This site has collected personal accounts of UFO sightings with data from 1900s to up to current day. These accounted for over 100,000 reports of UFO sightings.
 
- This dataset contained content including the time, location, duration, and other attributes in shape of object observed and a description of the event.

- The origianl data has records from 1959 – 2019, total records 88,096 rows of data.
 
- We’ve decided to keep only 9 years from 2010 – 2019 of most usable data to work with. This was 8830 rows of data which is more manageable.

- We will then scrape the NUFORC site for 2020 to make the record contain 10 years of cleaned data.

###  Read Data

In [2]:
# Dataset
ufo_data= pd.read_csv('data/nuforc_reports_2010_2019.csv')
ufo_data

summary             city  \
0     My wife was driving southeast on a fairly popu...          Chester   
1     I think that I may caught a UFO on the NBC Nig...       Rocky Hill   
2     I was driving towards the intersection of fall...           Ottawa   
3     The object has flashing lights that are green,...       Kirbyville   
4     Description is the same as Washington DC event...           Tucson   
...                                                 ...              ...   
8825           super low altitude stealth jet 9/1/11 NC  Franklin County   
8826  Light brown sphere found in the background of ...       Wolf Creek   
8827  Bun shaped object hovering over parking lot an...          Hamburg   
8828  Bright silent craft with no navigation lights ...         Falmouth   
8829      We noticed two bright lights in the dusk sky.           Linton   

     state  year            date_time     shape     duration  \
0       VA  2019  2019-12-12T18:43:00     light    5 seconds   
1       CT  2019  2019-03-22T18:30:00    circle  3-5 seconds   
2       ON  2019  2019-04-17T02:00:00  teardrop   10 seconds   
3       TX  2019  2019-04-02T20:25:00      disk   15 minutes   
4       AZ  2019  2019-05-01T11:00:00   unknown    5 minutes   
...    ...   ...                  ...       ...          ...   
8825    NC  2011  2011-09-01T09:00:00     other        i min   
8826    MT  2011  2011-09-01T10:25:00    sphere        short   
8827    NY  2011  2011-09-01T18:00:00      cone   20 seconds   
8828    MA  2011  2011-09-01T18:45:00     light    3 minutes   
8829    IN  2011  2011-09-01T20:50:00     light    3 minutes   

                                                  stats  \
0     Occurred : 12/12/2019 18:43  (Entered as : 12/...   
1     Occurred : 3/22/2019 18:30  (Entered as : 03/2...   
2     Occurred : 4/17/2019 02:00  (Entered as : 04-1...   
3     Occurred : 4/2/2019 20:25  (Entered as : 04/02...   
4     Occurred : 5/1/2019 11:00  (Entered as : 5/1/1...   
...                                                 ...   
8825  Occurred : 9/1/2011 09:00  (Entered as : 09/01...   
8826  Occurred : 9/1/2011 10:25  (Entered as : 09/01...   
8827  Occurred : 9/1/2011 18:00  (Entered as : 9/201...   
8828  Occurred : 9/1/2011 18:45  (Entered as : 09/01...   
8829  Occurred : 9/1/2011 20:50  (Entered as : 09/01...   

                                            report_link  \
0     http://www.nuforc.org/webreports/151/S151739.html   
1     http://www.nuforc.org/webreports/145/S145297.html   
2     http://www.nuforc.org/webreports/145/S145697.html   
3     http://www.nuforc.org/webreports/145/S145476.html   
4     http://www.nuforc.org/webreports/145/S145947.html   
...                                                 ...   
8825   http://www.nuforc.org/webreports/083/S83731.html   
8826   http://www.nuforc.org/webreports/084/S84138.html   
8827   http://www.nuforc.org/webreports/084/S84293.html   
8828   http://www.nuforc.org/webreports/083/S83636.html   
8829   http://www.nuforc.org/webreports/083/S83625.html   

                                                   text               posted  \
0     My wife was driving southeast on a fairly popu...  2019-12-22T00:00:00   
1     I think that I may caught a UFO on the NBC Nig...  2019-03-29T00:00:00   
2     I was driving towards the intersection of fall...  2019-04-18T00:00:00   
3     The object has flashing lights that are green,...  2019-04-08T00:00:00   
4     Description is the same as Washington, DC, eve...  2019-05-09T00:00:00   
...                                                 ...                  ...   
8825  super low altitude stealth jet 9/1/11 NC Extre...  2011-10-10T00:00:00   
8826  Light brown sphere found in the background of ...  2011-10-10T00:00:00   
8827  Bun shaped object hovering over parking lot an...  2011-10-10T00:00:00   
8828  Bright silent craft with no navigation lights ...  2011-10-10T00:00:00   
8829  We noticed two bright lights in the dusk sky. ...  2011-10-10T

###  Data Cleanup

In [65]:
# Drop summary column as it is identical to text and made data set diffuclt to work with
ufo_data = ufo_data.drop('summary',1)
ufo_data

city state  year            date_time     shape     duration  \
0             Chester    VA  2019  2019-12-12T18:43:00     light    5 seconds   
1          Rocky Hill    CT  2019  2019-03-22T18:30:00    circle  3-5 seconds   
2              Ottawa    ON  2019  2019-04-17T02:00:00  teardrop   10 seconds   
3          Kirbyville    TX  2019  2019-04-02T20:25:00      disk   15 minutes   
4              Tucson    AZ  2019  2019-05-01T11:00:00   unknown    5 minutes   
...               ...   ...   ...                  ...       ...          ...   
8825  Franklin County    NC  2011  2011-09-01T09:00:00     other        i min   
8826       Wolf Creek    MT  2011  2011-09-01T10:25:00    sphere        short   
8827          Hamburg    NY  2011  2011-09-01T18:00:00      cone   20 seconds   
8828         Falmouth    MA  2011  2011-09-01T18:45:00     light    3 minutes   
8829           Linton    IN  2011  2011-09-01T20:50:00     light    3 minutes   

                                                  stats  \
0     Occurred : 12/12/2019 18:43  (Entered as : 12/...   
1     Occurred : 3/22/2019 18:30  (Entered as : 03/2...   
2     Occurred : 4/17/2019 02:00  (Entered as : 04-1...   
3     Occurred : 4/2/2019 20:25  (Entered as : 04/02...   
4     Occurred : 5/1/2019 11:00  (Entered as : 5/1/1...   
...                                                 ...   
8825  Occurred : 9/1/2011 09:00  (Entered as : 09/01...   
8826  Occurred : 9/1/2011 10:25  (Entered as : 09/01...   
8827  Occurred : 9/1/2011 18:00  (Entered as : 9/201...   
8828  Occurred : 9/1/2011 18:45  (Entered as : 09/01...   
8829  Occurred : 9/1/2011 20:50  (Entered as : 09/01...   

                                            report_link  \
0     http://www.nuforc.org/webreports/151/S151739.html   
1     http://www.nuforc.org/webreports/145/S145297.html   
2     http://www.nuforc.org/webreports/145/S145697.html   
3     http://www.nuforc.org/webreports/145/S145476.html   
4     http://www.nuforc.org/webreports/145/S145947.html   
...                                                 ...   
8825   http://www.nuforc.org/webreports/083/S83731.html   
8826   http://www.nuforc.org/webreports/084/S84138.html   
8827   http://www.nuforc.org/webreports/084/S84293.html   
8828   http://www.nuforc.org/webreports/083/S83636.html   
8829   http://www.nuforc.org/webreports/083/S83625.html   

                                                   text               posted  \
0     My wife was driving southeast on a fairly popu...  2019-12-22T00:00:00   
1     I think that I may caught a UFO on the NBC Nig...  2019-03-29T00:00:00   
2     I was driving towards the intersection of fall...  2019-04-18T00:00:00   
3     The object has flashing lights that are green,...  2019-04-08T00:00:00   
4     Description is the same as Washington, DC, eve...  2019-05-09T00:00:00   
...                                                 ...                  ...   
8825  super low altitude stealth jet 9/1/11 NC Extre...  2011-10-10T00:00:00   
8826  Light brown sphere found in the background of ...  2011-10-10T00:00:00   
8827  Bun shaped object hovering over parking lot an...  2011-10-10T00:00:00   
8828  Bright silent craft with no navigation lights ...  2011-10-10T00:00:00   
8829  We noticed two bright lights in the dusk sky. ...  2011-10-10T00:00:00   

      city_latitude  city_longitude  
0         37.343152      -77.408582  
1         41.664800      -72.639300  
2         45.381383      -75.708501  
3         30.677200      -94.005200  
4         32.259941     -110.927542  
...             ...             ...  
8825            NaN             NaN  
8826            NaN             NaN  
8827      42.709100      -78.830100  
8828      41.574282      -70.627218  
8829      39.039600      -87.156500  

[8830 rows x 12 columns]

In [66]:
#convert into df
ufo_df = pd.DataFrame(ufo_data)
ufo_df

city state  year            date_time     shape     duration  \
0             Chester    VA  2019  2019-12-12T18:43:00     light    5 seconds   
1          Rocky Hill    CT  2019  2019-03-22T18:30:00    circle  3-5 seconds   
2              Ottawa    ON  2019  2019-04-17T02:00:00  teardrop   10 seconds   
3          Kirbyville    TX  2019  2019-04-02T20:25:00      disk   15 minutes   
4              Tucson    AZ  2019  2019-05-01T11:00:00   unknown    5 minutes   
...               ...   ...   ...                  ...       ...          ...   
8825  Franklin County    NC  2011  2011-09-01T09:00:00     other        i min   
8826       Wolf Creek    MT  2011  2011-09-01T10:25:00    sphere        short   
8827          Hamburg    NY  2011  2011-09-01T18:00:00      cone   20 seconds   
8828         Falmouth    MA  2011  2011-09-01T18:45:00     light    3 minutes   
8829           Linton    IN  2011  2011-09-01T20:50:00     light    3 minutes   

                                                  stats  \
0     Occurred : 12/12/2019 18:43  (Entered as : 12/...   
1     Occurred : 3/22/2019 18:30  (Entered as : 03/2...   
2     Occurred : 4/17/2019 02:00  (Entered as : 04-1...   
3     Occurred : 4/2/2019 20:25  (Entered as : 04/02...   
4     Occurred : 5/1/2019 11:00  (Entered as : 5/1/1...   
...                                                 ...   
8825  Occurred : 9/1/2011 09:00  (Entered as : 09/01...   
8826  Occurred : 9/1/2011 10:25  (Entered as : 09/01...   
8827  Occurred : 9/1/2011 18:00  (Entered as : 9/201...   
8828  Occurred : 9/1/2011 18:45  (Entered as : 09/01...   
8829  Occurred : 9/1/2011 20:50  (Entered as : 09/01...   

                                            report_link  \
0     http://www.nuforc.org/webreports/151/S151739.html   
1     http://www.nuforc.org/webreports/145/S145297.html   
2     http://www.nuforc.org/webreports/145/S145697.html   
3     http://www.nuforc.org/webreports/145/S145476.html   
4     http://www.nuforc.org/webreports/145/S145947.html   
...                                                 ...   
8825   http://www.nuforc.org/webreports/083/S83731.html   
8826   http://www.nuforc.org/webreports/084/S84138.html   
8827   http://www.nuforc.org/webreports/084/S84293.html   
8828   http://www.nuforc.org/webreports/083/S83636.html   
8829   http://www.nuforc.org/webreports/083/S83625.html   

                                                   text               posted  \
0     My wife was driving southeast on a fairly popu...  2019-12-22T00:00:00   
1     I think that I may caught a UFO on the NBC Nig...  2019-03-29T00:00:00   
2     I was driving towards the intersection of fall...  2019-04-18T00:00:00   
3     The object has flashing lights that are green,...  2019-04-08T00:00:00   
4     Description is the same as Washington, DC, eve...  2019-05-09T00:00:00   
...                                                 ...                  ...   
8825  super low altitude stealth jet 9/1/11 NC Extre...  2011-10-10T00:00:00   
8826  Light brown sphere found in the background of ...  2011-10-10T00:00:00   
8827  Bun shaped object hovering over parking lot an...  2011-10-10T00:00:00   
8828  Bright silent craft with no navigation lights ...  2011-10-10T00:00:00   
8829  We noticed two bright lights in the dusk sky. ...  2011-10-10T00:00:00   

      city_latitude  city_longitude  
0         37.343152      -77.408582  
1         41.664800      -72.639300  
2         45.381383      -75.708501  
3         30.677200      -94.005200  
4         32.259941     -110.927542  
...             ...             ...  
8825            NaN             NaN  
8826            NaN             NaN  
8827      42.709100      -78.830100  
8828      41.574282      -70.627218  
8829      39.039600      -87.156500  

[8830 rows x 12 columns]

In [67]:
#splitting date and time into two separate columns
ufo_df[['date','time']] = ufo_df.date_time.str.split("T",expand=True,)
ufo_df

city state  year            date_time     shape     duration  \
0             Chester    VA  2019  2019-12-12T18:43:00     light    5 seconds   
1          Rocky Hill    CT  2019  2019-03-22T18:30:00    circle  3-5 seconds   
2              Ottawa    ON  2019  2019-04-17T02:00:00  teardrop   10 seconds   
3          Kirbyville    TX  2019  2019-04-02T20:25:00      disk   15 minutes   
4              Tucson    AZ  2019  2019-05-01T11:00:00   unknown    5 minutes   
...               ...   ...   ...                  ...       ...          ...   
8825  Franklin County    NC  2011  2011-09-01T09:00:00     other        i min   
8826       Wolf Creek    MT  2011  2011-09-01T10:25:00    sphere        short   
8827          Hamburg    NY  2011  2011-09-01T18:00:00      cone   20 seconds   
8828         Falmouth    MA  2011  2011-09-01T18:45:00     light    3 minutes   
8829           Linton    IN  2011  2011-09-01T20:50:00     light    3 minutes   

                                                  stats  \
0     Occurred : 12/12/2019 18:43  (Entered as : 12/...   
1     Occurred : 3/22/2019 18:30  (Entered as : 03/2...   
2     Occurred : 4/17/2019 02:00  (Entered as : 04-1...   
3     Occurred : 4/2/2019 20:25  (Entered as : 04/02...   
4     Occurred : 5/1/2019 11:00  (Entered as : 5/1/1...   
...                                                 ...   
8825  Occurred : 9/1/2011 09:00  (Entered as : 09/01...   
8826  Occurred : 9/1/2011 10:25  (Entered as : 09/01...   
8827  Occurred : 9/1/2011 18:00  (Entered as : 9/201...   
8828  Occurred : 9/1/2011 18:45  (Entered as : 09/01...   
8829  Occurred : 9/1/2011 20:50  (Entered as : 09/01...   

                                            report_link  \
0     http://www.nuforc.org/webreports/151/S151739.html   
1     http://www.nuforc.org/webreports/145/S145297.html   
2     http://www.nuforc.org/webreports/145/S145697.html   
3     http://www.nuforc.org/webreports/145/S145476.html   
4     http://www.nuforc.org/webreports/145/S145947.html   
...                                                 ...   
8825   http://www.nuforc.org/webreports/083/S83731.html   
8826   http://www.nuforc.org/webreports/084/S84138.html   
8827   http://www.nuforc.org/webreports/084/S84293.html   
8828   http://www.nuforc.org/webreports/083/S83636.html   
8829   http://www.nuforc.org/webreports/083/S83625.html   

                                                   text               posted  \
0     My wife was driving southeast on a fairly popu...  2019-12-22T00:00:00   
1     I think that I may caught a UFO on the NBC Nig...  2019-03-29T00:00:00   
2     I was driving towards the intersection of fall...  2019-04-18T00:00:00   
3     The object has flashing lights that are green,...  2019-04-08T00:00:00   
4     Description is the same as Washington, DC, eve...  2019-05-09T00:00:00   
...                                                 ...                  ...   
8825  super low altitude stealth jet 9/1/11 NC Extre...  2011-10-10T00:00:00   
8826  Light brown sphere found in the background of ...  2011-10-10T00:00:00   
8827  Bun shaped object hovering over parking lot an...  2011-10-10T00:00:00   
8828  Bright silent craft with no navigation lights ...  2011-10-10T00:00:00   
8829  We noticed two bright lights in the dusk sky. ...  2011-10-10T00:00:00   

      city_latitude  city_longitude        date      time  
0         37.343152      -77.408582  2019-12-12  18:43:00  
1         41.664800      -72.639300  2019-03-22  18:30:00  
2         45.381383      -75.708501  2019-04-17  02:00:00  
3         30.677200      -94.005200  2019-04-02  20:25:00  
4         32.259941     -110.927542  2019-05-01  11:00:00  
...             ...             ...         ...       ...  
8825            NaN             NaN  2011-09-01  09:00:00  
8826            NaN             NaN  2011-09-01  10:25:00  
8827      42.709100      -78.830100  2011-09-01  18:00:00  
8828      41.574282      -70.627218  2011-09-01  18:45:00  
8829 

In [68]:
# convert to certain columns into type datetime format
ufo_df['date_time'] = pd.to_datetime(ufo_df['date_time'], format='%Y-%m-%dT%H:%M:%S.%f')
ufo_df['date'] = pd.to_datetime(ufo_df['date'], format='%Y-%m-%d')
ufo_df['time'] = pd.to_datetime(ufo_df['time'], format='%H:%M:%S')
ufo_df.dtypes

city                      object
state                     object
year                       int64
date_time         datetime64[ns]
shape                     object
duration                  object
stats                     object
report_link               object
text                      object
posted                    object
city_latitude            float64
city_longitude           float64
date              datetime64[ns]
time              datetime64[ns]
dtype: object

In [69]:
#convert nan values into empty strings
ufo_df = ufo_df.replace(np.nan, '', regex=True)
ufo_df

city state  year           date_time     shape     duration  \
0             Chester    VA  2019 2019-12-12 18:43:00     light    5 seconds   
1          Rocky Hill    CT  2019 2019-03-22 18:30:00    circle  3-5 seconds   
2              Ottawa    ON  2019 2019-04-17 02:00:00  teardrop   10 seconds   
3          Kirbyville    TX  2019 2019-04-02 20:25:00      disk   15 minutes   
4              Tucson    AZ  2019 2019-05-01 11:00:00   unknown    5 minutes   
...               ...   ...   ...                 ...       ...          ...   
8825  Franklin County    NC  2011 2011-09-01 09:00:00     other        i min   
8826       Wolf Creek    MT  2011 2011-09-01 10:25:00    sphere        short   
8827          Hamburg    NY  2011 2011-09-01 18:00:00      cone   20 seconds   
8828         Falmouth    MA  2011 2011-09-01 18:45:00     light    3 minutes   
8829           Linton    IN  2011 2011-09-01 20:50:00     light    3 minutes   

                                                  stats  \
0     Occurred : 12/12/2019 18:43  (Entered as : 12/...   
1     Occurred : 3/22/2019 18:30  (Entered as : 03/2...   
2     Occurred : 4/17/2019 02:00  (Entered as : 04-1...   
3     Occurred : 4/2/2019 20:25  (Entered as : 04/02...   
4     Occurred : 5/1/2019 11:00  (Entered as : 5/1/1...   
...                                                 ...   
8825  Occurred : 9/1/2011 09:00  (Entered as : 09/01...   
8826  Occurred : 9/1/2011 10:25  (Entered as : 09/01...   
8827  Occurred : 9/1/2011 18:00  (Entered as : 9/201...   
8828  Occurred : 9/1/2011 18:45  (Entered as : 09/01...   
8829  Occurred : 9/1/2011 20:50  (Entered as : 09/01...   

                                            report_link  \
0     http://www.nuforc.org/webreports/151/S151739.html   
1     http://www.nuforc.org/webreports/145/S145297.html   
2     http://www.nuforc.org/webreports/145/S145697.html   
3     http://www.nuforc.org/webreports/145/S145476.html   
4     http://www.nuforc.org/webreports/145/S145947.html   
...                                                 ...   
8825   http://www.nuforc.org/webreports/083/S83731.html   
8826   http://www.nuforc.org/webreports/084/S84138.html   
8827   http://www.nuforc.org/webreports/084/S84293.html   
8828   http://www.nuforc.org/webreports/083/S83636.html   
8829   http://www.nuforc.org/webreports/083/S83625.html   

                                                   text               posted  \
0     My wife was driving southeast on a fairly popu...  2019-12-22T00:00:00   
1     I think that I may caught a UFO on the NBC Nig...  2019-03-29T00:00:00   
2     I was driving towards the intersection of fall...  2019-04-18T00:00:00   
3     The object has flashing lights that are green,...  2019-04-08T00:00:00   
4     Description is the same as Washington, DC, eve...  2019-05-09T00:00:00   
...                                                 ...                  ...   
8825  super low altitude stealth jet 9/1/11 NC Extre...  2011-10-10T00:00:00   
8826  Light brown sphere found in the background of ...  2011-10-10T00:00:00   
8827  Bun shaped object hovering over parking lot an...  2011-10-10T00:00:00   
8828  Bright silent craft with no navigation lights ...  2011-10-10T00:00:00   
8829  We noticed two bright lights in the dusk sky. ...  2011-10-10T00:00:00   

     city_latitude city_longitude       date                time  
0          37.3432       -77.4086 2019-12-12 1900-01-01 18:43:00  
1          41.6648       -72.6393 2019-03-22 1900-01-01 18:30:00  
2          45.3814       -75.7085 2019-04-17 1900-01-01 02:00:00  
3          30.6772       -94.0052 2019-04-02 1900-01-01 20:25:00  
4          32.2599       -110.928 2019-05-01 1900-01-01 11:00:00  
...            ...            ...        ...                 ...  
8825                              2011-09-01 1900-01-01 09:00:00  
8826                              2011-09-01 1900-01-01 10:25:00  
8827       42.7091       -78.8301 2011-09-01 1900-01-01 18:00:00  
8828   

In [70]:
ufo_df.dtypes # identify data types

city                      object
state                     object
year                       int64
date_time         datetime64[ns]
shape                     object
duration                  object
stats                     object
report_link               object
text                      object
posted                    object
city_latitude             object
city_longitude            object
date              datetime64[ns]
time              datetime64[ns]
dtype: object

## Spliting Main Dataframe into two
- As one has geo coordinates (US centric)
- And the other which we are missing geo coordinates (non US locations)

In [71]:
# filter out missing coordinates
# where coordinates is blank
ufo_missing_coordinates = ufo_df.loc[ufo_df['city_latitude'] == ""]
ufo_missing_coordinates

city state  year           date_time     shape  \
6     Caloocan City (Philippines)        2019 2019-06-06 19:00:00     other   
13                         Leyner    CO  2019 2019-08-09 20:22:00     light   
14                       Catalina    AZ  2019 2019-08-09 20:30:00     cigar   
18                   Marin County    CA  2019 2019-08-01 22:30:00    sphere   
21               West Bowmanstown    PA  2019 2019-06-27 01:00:00  triangle   
...                           ...   ...   ...                 ...       ...   
8793                   East Sooke    BC  2011 2011-10-27 19:30:00     light   
8795              Upper Peninsula    MI  2011 2011-10-27 21:00:00     light   
8808                     Hazelton    KS  2011 2011-08-31 21:52:00   unknown   
8825              Franklin County    NC  2011 2011-09-01 09:00:00     other   
8826                   Wolf Creek    MT  2011 2011-09-01 10:25:00    sphere   

                       duration  \
6                19:00 to 19:30   
13                   20 minutes   
14                   40 seconds   
18                    3 minutes   
21    First 3min seconed 3 min.   
...                         ...   
8793                    >5 min.   
8795                  5 minutes   
8808                       2min   
8825                      i min   
8826                      short   

                                                  stats  \
6     Occurred : 6/6/2019 19:00  (Entered as : 6/6/2...   
13    Occurred : 8/9/2019 20:22  (Entered as : 08/09...   
14    Occurred : 8/9/2019 20:30  (Entered as : 08/09...   
18    Occurred : 8/1/2019 22:30  (Entered as : 8-01-...   
21    Occurred : 6/27/2019 01:00  (Entered as : 06/ ...   
...                                                 ...   
8793  Occurred : 10/27/2011 19:30  (Entered as : 10/...   
8795  Occurred : 10/27/2011 21:00  (Entered as : 10/...   
8808  Occurred : 8/31/2011 21:52  (Entered as : 08/3...   
8825  Occurred : 9/1/2011 09:00  (Entered as : 09/01...   
8826  Occurred : 9/1/2011 10:25  (Entered as : 09/01...   

                                            report_link  \
6     http://www.nuforc.org/webreports/146/S146694.html   
13    http://www.nuforc.org/webreports/148/S148205.html   
14    http://www.nuforc.org/webreports/148/S148290.html   
18    http://www.nuforc.org/webreports/148/S148424.html   
21    http://www.nuforc.org/webreports/148/S148370.html   
...                                                 ...   
8793   http://www.nuforc.org/webreports/085/S85034.html   
8795   http://www.nuforc.org/webreports/084/S84955.html   
8808   http://www.nuforc.org/webreports/083/S83613.html   
8825   http://www.nuforc.org/webreports/083/S83731.html   
8826   http://www.nuforc.org/webreports/084/S84138.html   

                                                   text               posted  \
6     group of lights formation sweeping thru a nigh...  2019-06-07T00:00:00   
13    There where 4 bright lights in a shape of a di...  2019-08-23T00:00:00   
14    My wife and I were taking our usual evening st...  2019-08-23T00:00:00   
18    7 stationary brightly lit objects seen  over h...  2019-08-23T00:00:00   
21    Spotted it behind my property next to Rt 476 N...  2019-08-23T00:00:00   
...                                                 ...                  ...   
8793  Bright white/blue light moving in fluid manner...  2011-12-12T00:00:00   
8795  Two brilliant red lights, side-by-side, moved ...  2011-12-12T00:00:00   
8808  bright white light in rural Barber county kans...  2011-10-10T00:00:00   
8825  super low altitude stealth jet 9/1/11 NC Extre...  2011-10-10T00:00:00   
8826  Light brown sphere found in the background of ...  2011-10-10T00:00:00   

     city_latitude city_longitude       date                time  
6                                 2019-06-06 1900-01-01 19:00:00  
13                                2019-08-09 1900-01-01 20:22:00  
14                                2019-08-09 1900-01-01 20:30:00  
18          

In [73]:
# Convert this into a separate dataframe
ufo_missing_coordinates = pd.DataFrame(ufo_missing_coordinates)
ufo_missing_coordinates.head()

city state  year           date_time     shape  \
6   Caloocan City (Philippines)        2019 2019-06-06 19:00:00     other   
13                       Leyner    CO  2019 2019-08-09 20:22:00     light   
14                     Catalina    AZ  2019 2019-08-09 20:30:00     cigar   
18                 Marin County    CA  2019 2019-08-01 22:30:00    sphere   
21             West Bowmanstown    PA  2019 2019-06-27 01:00:00  triangle   

                     duration  \
6              19:00 to 19:30   
13                 20 minutes   
14                 40 seconds   
18                  3 minutes   
21  First 3min seconed 3 min.   

                                                stats  \
6   Occurred : 6/6/2019 19:00  (Entered as : 6/6/2...   
13  Occurred : 8/9/2019 20:22  (Entered as : 08/09...   
14  Occurred : 8/9/2019 20:30  (Entered as : 08/09...   
18  Occurred : 8/1/2019 22:30  (Entered as : 8-01-...   
21  Occurred : 6/27/2019 01:00  (Entered as : 06/ ...   

                                          report_link  \
6   http://www.nuforc.org/webreports/146/S146694.html   
13  http://www.nuforc.org/webreports/148/S148205.html   
14  http://www.nuforc.org/webreports/148/S148290.html   
18  http://www.nuforc.org/webreports/148/S148424.html   
21  http://www.nuforc.org/webreports/148/S148370.html   

                                                 text               posted  \
6   group of lights formation sweeping thru a nigh...  2019-06-07T00:00:00   
13  There where 4 bright lights in a shape of a di...  2019-08-23T00:00:00   
14  My wife and I were taking our usual evening st...  2019-08-23T00:00:00   
18  7 stationary brightly lit objects seen  over h...  2019-08-23T00:00:00   
21  Spotted it behind my property next to Rt 476 N...  2019-08-23T00:00:00   

   city_latitude city_longitude       date                time  
6                               2019-06-06 1900-01-01 19:00:00  
13                              2019-08-09 1900-01-01 20:22:00  
14                              2019-08-09 1900-01-01 20:30:00  
18                              2019-08-01 1900-01-01 22:30:00  
21                              2019-06-27 1900-01-01 01:00:00

In [75]:
# remove all missing coordinates rows from main df so we can merge it again later
# create a separate dataframe that has already have all the required information 
ufo_usa_only = ufo_df[ufo_df.city_latitude != ""]
ufo_usa_only = pd.DataFrame(ufo_usa_only)
ufo_usa_only

city state  year           date_time     shape     duration  \
0           Chester    VA  2019 2019-12-12 18:43:00     light    5 seconds   
1        Rocky Hill    CT  2019 2019-03-22 18:30:00    circle  3-5 seconds   
2            Ottawa    ON  2019 2019-04-17 02:00:00  teardrop   10 seconds   
3        Kirbyville    TX  2019 2019-04-02 20:25:00      disk   15 minutes   
4            Tucson    AZ  2019 2019-05-01 11:00:00   unknown    5 minutes   
...             ...   ...   ...                 ...       ...          ...   
8823      Bremerton    WA  2011 2011-09-01 03:00:00    circle   30 Minutes   
8824  Friday Harbor    WA  2011 2011-09-01 04:00:00    circle       45 min   
8827        Hamburg    NY  2011 2011-09-01 18:00:00      cone   20 seconds   
8828       Falmouth    MA  2011 2011-09-01 18:45:00     light    3 minutes   
8829         Linton    IN  2011 2011-09-01 20:50:00     light    3 minutes   

                                                  stats  \
0     Occurred : 12/12/2019 18:43  (Entered as : 12/...   
1     Occurred : 3/22/2019 18:30  (Entered as : 03/2...   
2     Occurred : 4/17/2019 02:00  (Entered as : 04-1...   
3     Occurred : 4/2/2019 20:25  (Entered as : 04/02...   
4     Occurred : 5/1/2019 11:00  (Entered as : 5/1/1...   
...                                                 ...   
8823  Occurred : 9/1/2011 03:00  (Entered as : 09/01...   
8824  Occurred : 9/1/2011 04:00  (Entered as : 9-1-2...   
8827  Occurred : 9/1/2011 18:00  (Entered as : 9/201...   
8828  Occurred : 9/1/2011 18:45  (Entered as : 09/01...   
8829  Occurred : 9/1/2011 20:50  (Entered as : 09/01...   

                                            report_link  \
0     http://www.nuforc.org/webreports/151/S151739.html   
1     http://www.nuforc.org/webreports/145/S145297.html   
2     http://www.nuforc.org/webreports/145/S145697.html   
3     http://www.nuforc.org/webreports/145/S145476.html   
4     http://www.nuforc.org/webreports/145/S145947.html   
...                                                 ...   
8823   http://www.nuforc.org/webreports/084/S84105.html   
8824   http://www.nuforc.org/webreports/083/S83667.html   
8827   http://www.nuforc.org/webreports/084/S84293.html   
8828   http://www.nuforc.org/webreports/083/S83636.html   
8829   http://www.nuforc.org/webreports/083/S83625.html   

                                                   text               posted  \
0     My wife was driving southeast on a fairly popu...  2019-12-22T00:00:00   
1     I think that I may caught a UFO on the NBC Nig...  2019-03-29T00:00:00   
2     I was driving towards the intersection of fall...  2019-04-18T00:00:00   
3     The object has flashing lights that are green,...  2019-04-08T00:00:00   
4     Description is the same as Washington, DC, eve...  2019-05-09T00:00:00   
...                                                 ...                  ...   
8823  I was smoking a cigarette on my front porch wi...  2011-10-10T00:00:00   
8824  Bright light and a deep rumbling sound with ot...  2011-10-10T00:00:00   
8827  Bun shaped object hovering over parking lot an...  2011-10-10T00:00:00   
8828  Bright silent craft with no navigation lights ...  2011-10-10T00:00:00   
8829  We noticed two bright lights in the dusk sky. ...  2011-10-10T00:00:00   

     city_latitude city_longitude       date                time  
0          37.3432       -77.4086 2019-12-12 1900-01-01 18:43:00  
1          41.6648       -72.6393 2019-03-22 1900-01-01 18:30:00  
2          45.3814       -75.7085 2019-04-17 1900-01-01 02:00:00  
3          30.6772       -94.0052 2019-04-02 1900-01-01 20:25:00  
4          32.2599       -110.928 2019-05-01 1900-01-01 11:00:00  
...            ...            ...        ...                 ...  
8823       47.5993        -122.67 2011-09-01 1900-01-01 03:00:00  
8824       48.5372       -123.068 2011-09-01 1900-01-01 04:00:00  
8827       42.7091       -78.8301 2011-09-01 1900-01-01 18:00:00  
8828       41.5743       -70.

## Find Missing Geo Coordinates
- A number of locations (1558 rows) in our data set has missing geo coordinates
- However they do have a city name, and we can try to find the geo coordinates for what we can
- This was done using Open Cage Data API https://opencagedata.com/

In [77]:
# Review dataframe with missing coordinates
ufo_missing_coordinates.head()

city state  year           date_time     shape  \
6   Caloocan City (Philippines)        2019 2019-06-06 19:00:00     other   
13                       Leyner    CO  2019 2019-08-09 20:22:00     light   
14                     Catalina    AZ  2019 2019-08-09 20:30:00     cigar   
18                 Marin County    CA  2019 2019-08-01 22:30:00    sphere   
21             West Bowmanstown    PA  2019 2019-06-27 01:00:00  triangle   

                     duration  \
6              19:00 to 19:30   
13                 20 minutes   
14                 40 seconds   
18                  3 minutes   
21  First 3min seconed 3 min.   

                                                stats  \
6   Occurred : 6/6/2019 19:00  (Entered as : 6/6/2...   
13  Occurred : 8/9/2019 20:22  (Entered as : 08/09...   
14  Occurred : 8/9/2019 20:30  (Entered as : 08/09...   
18  Occurred : 8/1/2019 22:30  (Entered as : 8-01-...   
21  Occurred : 6/27/2019 01:00  (Entered as : 06/ ...   

                                          report_link  \
6   http://www.nuforc.org/webreports/146/S146694.html   
13  http://www.nuforc.org/webreports/148/S148205.html   
14  http://www.nuforc.org/webreports/148/S148290.html   
18  http://www.nuforc.org/webreports/148/S148424.html   
21  http://www.nuforc.org/webreports/148/S148370.html   

                                                 text               posted  \
6   group of lights formation sweeping thru a nigh...  2019-06-07T00:00:00   
13  There where 4 bright lights in a shape of a di...  2019-08-23T00:00:00   
14  My wife and I were taking our usual evening st...  2019-08-23T00:00:00   
18  7 stationary brightly lit objects seen  over h...  2019-08-23T00:00:00   
21  Spotted it behind my property next to Rt 476 N...  2019-08-23T00:00:00   

   city_latitude city_longitude       date                time  
6                               2019-06-06 1900-01-01 19:00:00  
13                              2019-08-09 1900-01-01 20:22:00  
14                              2019-08-09 1900-01-01 20:30:00  
18                              2019-08-01 1900-01-01 22:30:00  
21                              2019-06-27 1900-01-01 01:00:00

In [78]:
# Set the index to city so that we can do our API search based on the index
ufo_missing_coordinates.set_index("city", inplace = True)
ufo_missing_coordinates.head()

state  year           date_time     shape  \
city                                                                    
Caloocan City (Philippines)        2019 2019-06-06 19:00:00     other   
Leyner                         CO  2019 2019-08-09 20:22:00     light   
Catalina                       AZ  2019 2019-08-09 20:30:00     cigar   
Marin County                   CA  2019 2019-08-01 22:30:00    sphere   
West Bowmanstown               PA  2019 2019-06-27 01:00:00  triangle   

                                              duration  \
city                                                     
Caloocan City (Philippines)             19:00 to 19:30   
Leyner                                      20 minutes   
Catalina                                    40 seconds   
Marin County                                 3 minutes   
West Bowmanstown             First 3min seconed 3 min.   

                                                                         stats  \
city                                                                             
Caloocan City (Philippines)  Occurred : 6/6/2019 19:00  (Entered as : 6/6/2...   
Leyner                       Occurred : 8/9/2019 20:22  (Entered as : 08/09...   
Catalina                     Occurred : 8/9/2019 20:30  (Entered as : 08/09...   
Marin County                 Occurred : 8/1/2019 22:30  (Entered as : 8-01-...   
West Bowmanstown             Occurred : 6/27/2019 01:00  (Entered as : 06/ ...   

                                                                   report_link  \
city                                                                             
Caloocan City (Philippines)  http://www.nuforc.org/webreports/146/S146694.html   
Leyner                       http://www.nuforc.org/webreports/148/S148205.html   
Catalina                     http://www.nuforc.org/webreports/148/S148290.html   
Marin County                 http://www.nuforc.org/webreports/148/S148424.html   
West Bowmanstown             http://www.nuforc.org/webreports/148/S148370.html   

                                                                          text  \
city                                                                             
Caloocan City (Philippines)  group of lights formation sweeping thru a nigh...   
Leyner                       There where 4 bright lights in a shape of a di...   
Catalina                     My wife and I were taking our usual evening st...   
Marin County                 7 stationary brightly lit objects seen  over h...   
West Bowmanstown             Spotted it behind my property next to Rt 476 N...   

                                          posted city_latitude city_longitude  \
city                                                                            
Caloocan City (Philippines)  2019-06-07T00:00:00                                
Leyner                       2019-08-23T00:00:00                                
Catalina                     2019-08-23T00:00:00                                
Marin County                 2019-08-23T00:00:00                                
West Bowmanstown             2019-08-23T00:00:00                                

                                  date                time  
city                                                        
Caloocan City (Philippines) 2019-06-06 1900-01-01 19:00:00  
Leyner                      2019-08-09 1900-01-01 20:22:00  
Catalina                    2019-08-09 1900-01-01 20:30:00  
Marin County                2019-08-01 1900-01-01 22:30:00  
West Bowmanstown            2019-06-27 1900-01-01 01:00:00

In [136]:
# Import our API requirements
key = key
geocoder = OpenCageGeocode(key)
addressfile = ufo_missing_coordinates

In [80]:
# run the API to extract Coordinates
#set up a coordiantes list just in case
coordinates = []
for line in ufo_missing_coordinates.index:
    try: 
      address = line.strip()
      results = geocoder.geocode(address, no_annotations='1')

      if results and len(results):
        city  = results[0]['formatted']
        longitude = results[0]['geometry']['lng']
        latitude  = results[0]['geometry']['lat']        
        
        #loop through and find missing coordinates and append it to dataframe 
        ufo_missing_coordinates.loc[line,"city_latitude"] = results[0]['geometry']['lng']
        ufo_missing_coordinates.loc[line,"city_longitude"] = results[0]['geometry']['lat']
        
        #get response 
        response_output = {
            "city": city,
            "city_latitude":latitude,
            "city_longitude":longitude
                    }
        #also append coordinates list
        coordinates.append(response_output)
        
        print(u'%f;%f;%s' % (latitude, longitude, address))
      else:
        #if not found append the field as blank
        sys.stderr.write("not found: %s\n" % address)
        ufo_missing_coordinates.loc[line,"city_latitude"] = ""
        ufo_missing_coordinates.loc[line,"city_longitude"] = ""
    #handling error exceptions
    except (IOError, AttributeError):
        print('Error: File %s does not appear to exist.' % addressfile)
        # Open cage API only allows 2500 API call a day so once limit is hit notify
    except RateLimitExceededError as ex:
      print(ex)
    continue

14.651322;120.972296;Caloocan City (Philippines)
40.049707;-105.102484;Leyner
32.482131;-110.900197;Catalina
38.040914;-122.619964;Marin County
40.800047;-75.664340;West Bowmanstown
43.992421;-102.239981;Wall
43.601077;-86.337943;Shelby Twp
45.766911;11.734347;Bassano del Grappa (Italy)
9.010793;38.761252;Addis Ababa (Ethiopia)
40.995375;-74.012083;River Vale
37.782883;-81.990124;Micco
42.575186;-83.293981;Bloomfield Twp.
30.173419;-95.504686;The Woodlands
28.261679;-81.618408;Champions Gate
35.173930;33.364726;Nicosia
31.927907;-81.096552;Mc Alpin
37.201902;-113.187854;Near Virgin, UT
47.214055;-94.753310;LaPorte
-26.250000;28.400000;Sabdy Springs
39.567551;-84.816067;West College Corner
48.509465;-101.885162;Donnybrook
46.500283;-66.750183;New Brunswick


not found: 


45.148906;-93.963920;Wright
53.407199;-2.991680;Liverpool (UK/England)
48.700887;-121.099227;Diablo Lake
37.211334;-98.703326;Barber
40.749824;-73.797634;Queens
39.108930;-105.756164;Park County
38.833958;-104.825349;Colorado Springs?
41.675927;-74.072643;Ardonia
39.288463;-76.567545;Highlandtown Lake
52.921262;-1.476149;Derby
37.079179;-106.008633;Antonito
-45.000000;-15.000000;Atlantic Ocean (240 miles E of FL)
39.364285;-74.422935;Atlantic City??
42.980070;-75.250997;Clayville
31.807297;-102.241338;Estes
40.026489;-119.569067;Pyramid Lake
34.365169;-77.630529;Topsail Beach
34.490167;-77.431630;North Topsail Beach
32.274591;-90.132031;Pearl
39.658735;-74.168292;Surf City
39.843377;-86.397774;Brownsburg
30.381580;-97.761873;Adirondack
48.082194;17.263614;Dunajska Luzna (Slovakia)
33.567315;-111.523569;Saguaro Lake
38.355862;-104.711211;Pueblo West
47.755883;-122.341043;Shoreline
47.657110;-117.261394;Spokane Valley
-26.304490;-48.848673;Joinville (Brazil)
43.182535;-94.209098;Kossuth


not found: Monnroe


35.584385;-78.799869;Fuquay Varina
43.020445;-71.600257;Goffstown
39.869552;-104.985181;Thornton
44.468583;-109.437631;Wapiti
41.367591;-75.062395;Lords Valley


...

38.249880;-85.635742;Saint Matthew's
39.302635;-96.770934;Riley
32.930400;-95.574960;Emory-Yantis
52.402145;-1.501994;Coventry (UK/England)
25.238865;55.301987;in space
47.004876;-108.346376;Winnett
37.528443;-81.944009;War Eagle


not found: Television program


47.784542;-122.274295;Brier
44.326084;-76.027327;Wellesley Island
40.607774;-95.291649;Northboro
39.901781;-74.741827;Beaverville
41.343953;-122.969655;Albert Lake
43.653482;-79.383935;Toronto (Canada)
37.204110;-119.058170;Dinkey Lakes Wilderness


not found: 


45.415103;-73.677344;Kahnawake
18.255565;-65.972380;Gurabo (Puerto Rico)
1.357107;103.819499;Singapore
43.606776;-72.979288;Rutland City
51.530484;-0.118550;Field Store
41.188130;-103.699765;Kimball
43.126872;0.351334;LaGrange
37.557760;-89.030242;Interstate 40
48.943769;2.034175;Carrieres sous Poissy (France)
21.316428;-158.069566;Kalaeloa
51.933470;0.247960;Henham (near Stansted Airport) (UK/England)
45.917355;-119.342518;Umatilla
18.121562;-65.457464;Vieques (Puerto Rico)
51.171689;-0.164075;Horley (UK/England)
51.332610;-0.267821;Epsom (Surrey) (UK/England)
52.372760;4.893604;Amsterdam (Netherlands)
26.271192;-80.270604;Coral Springs
37.896870;-122.508863;Strawberry
13.951152;-91.301008;Salt Flats
52.544248;0.088066;Saint Petersburg Causeway
26.166689;-80.278757;Sunrise
40.903987;-73.197890;Nissequogue
23.059836;-109.702515;San Jose del Cabo (Mexico)
-38.149281;144.359827;Geelong (Australia)
33.891774;-87.526124;Carbon Hill
55.824029;-5.081406;Isle of Bute (UK/Scotland)
45.393853;-

In [92]:
# reset index of df
ufo_missing_coordinates.reset_index(inplace=True)
ufo_missing_coordinates.head()

index                         city state  year           date_time  \
0      0  Caloocan City (Philippines)        2019 2019-06-06 19:00:00   
1      1                       Leyner    CO  2019 2019-08-09 20:22:00   
2      2                     Catalina    AZ  2019 2019-08-09 20:30:00   
3      3                 Marin County    CA  2019 2019-08-01 22:30:00   
4      4             West Bowmanstown    PA  2019 2019-06-27 01:00:00   

      shape                   duration  \
0     other             19:00 to 19:30   
1     light                 20 minutes   
2     cigar                 40 seconds   
3    sphere                  3 minutes   
4  triangle  First 3min seconed 3 min.   

                                               stats  \
0  Occurred : 6/6/2019 19:00  (Entered as : 6/6/2...   
1  Occurred : 8/9/2019 20:22  (Entered as : 08/09...   
2  Occurred : 8/9/2019 20:30  (Entered as : 08/09...   
3  Occurred : 8/1/2019 22:30  (Entered as : 8-01-...   
4  Occurred : 6/27/2019 01:00  (Entered as : 06/ ...   

                                         report_link  \
0  http://www.nuforc.org/webreports/146/S146694.html   
1  http://www.nuforc.org/webreports/148/S148205.html   
2  http://www.nuforc.org/webreports/148/S148290.html   
3  http://www.nuforc.org/webreports/148/S148424.html   
4  http://www.nuforc.org/webreports/148/S148370.html   

                                                text               posted  \
0  group of lights formation sweeping thru a nigh...  2019-06-07T00:00:00   
1  There where 4 bright lights in a shape of a di...  2019-08-23T00:00:00   
2  My wife and I were taking our usual evening st...  2019-08-23T00:00:00   
3  7 stationary brightly lit objects seen  over h...  2019-08-23T00:00:00   
4  Spotted it behind my property next to Rt 476 N...  2019-08-23T00:00:00   

  city_latitude city_longitude       date                time  
0       120.972        14.6513 2019-06-06 1900-01-01 19:00:00  
1      -105.102        40.0497 2019-08-09 1900-01-01 20:22:00  
2        -110.9        32.4821 2019-08-09 1900-01-01 20:30:00  
3       -122.62        38.0409 2019-08-01 1900-01-01 22:30:00  
4      -75.6643           40.8 2019-06-27 1900-01-01 01:00:00

In [93]:
# Drop index column 
#ufo_missing_coordinates = ufo_missing_coordinates.drop('index',1)
ufo_missing_coordinates.dtypes

index                      int64
city                      object
state                     object
year                       int64
date_time         datetime64[ns]
shape                     object
duration                  object
stats                     object
report_link               object
text                      object
posted                    object
city_latitude             object
city_longitude            object
date              datetime64[ns]
time              datetime64[ns]
dtype: object

## Joining the two Dataframe back together
- Once we have all the geo coordinates we join the data back together to make a single dataframe

In [91]:
# lets merge US data and international data together into final dataframe
ufo_final_df = pd.concat([ufo_usa_only, ufo_missing_coordinates], axis=0)
ufo_final_df

city state  year           date_time     shape     duration  \
0             Chester    VA  2019 2019-12-12 18:43:00     light    5 seconds   
1          Rocky Hill    CT  2019 2019-03-22 18:30:00    circle  3-5 seconds   
2              Ottawa    ON  2019 2019-04-17 02:00:00  teardrop   10 seconds   
3          Kirbyville    TX  2019 2019-04-02 20:25:00      disk   15 minutes   
4              Tucson    AZ  2019 2019-05-01 11:00:00   unknown    5 minutes   
...               ...   ...   ...                 ...       ...          ...   
1572       East Sooke    BC  2011 2011-10-27 19:30:00     light      >5 min.   
1573  Upper Peninsula    MI  2011 2011-10-27 21:00:00     light    5 minutes   
1574         Hazelton    KS  2011 2011-08-31 21:52:00   unknown         2min   
1575  Franklin County    NC  2011 2011-09-01 09:00:00     other        i min   
1576       Wolf Creek    MT  2011 2011-09-01 10:25:00    sphere        short   

                                                  stats  \
0     Occurred : 12/12/2019 18:43  (Entered as : 12/...   
1     Occurred : 3/22/2019 18:30  (Entered as : 03/2...   
2     Occurred : 4/17/2019 02:00  (Entered as : 04-1...   
3     Occurred : 4/2/2019 20:25  (Entered as : 04/02...   
4     Occurred : 5/1/2019 11:00  (Entered as : 5/1/1...   
...                                                 ...   
1572  Occurred : 10/27/2011 19:30  (Entered as : 10/...   
1573  Occurred : 10/27/2011 21:00  (Entered as : 10/...   
1574  Occurred : 8/31/2011 21:52  (Entered as : 08/3...   
1575  Occurred : 9/1/2011 09:00  (Entered as : 09/01...   
1576  Occurred : 9/1/2011 10:25  (Entered as : 09/01...   

                                            report_link  \
0     http://www.nuforc.org/webreports/151/S151739.html   
1     http://www.nuforc.org/webreports/145/S145297.html   
2     http://www.nuforc.org/webreports/145/S145697.html   
3     http://www.nuforc.org/webreports/145/S145476.html   
4     http://www.nuforc.org/webreports/145/S145947.html   
...                                                 ...   
1572   http://www.nuforc.org/webreports/085/S85034.html   
1573   http://www.nuforc.org/webreports/084/S84955.html   
1574   http://www.nuforc.org/webreports/083/S83613.html   
1575   http://www.nuforc.org/webreports/083/S83731.html   
1576   http://www.nuforc.org/webreports/084/S84138.html   

                                                   text               posted  \
0     My wife was driving southeast on a fairly popu...  2019-12-22T00:00:00   
1     I think that I may caught a UFO on the NBC Nig...  2019-03-29T00:00:00   
2     I was driving towards the intersection of fall...  2019-04-18T00:00:00   
3     The object has flashing lights that are green,...  2019-04-08T00:00:00   
4     Description is the same as Washington, DC, eve...  2019-05-09T00:00:00   
...                                                 ...                  ...   
1572  Bright white/blue light moving in fluid manner...  2011-12-12T00:00:00   
1573  Two brilliant red lights, side-by-side, moved ...  2011-12-12T00:00:00   
1574  bright white light in rural Barber county kans...  2011-10-10T00:00:00   
1575  super low altitude stealth jet 9/1/11 NC Extre...  2011-10-10T00:00:00   
1576  Light brown sphere found in the background of ...  2011-10-10T00:00:00   

     city_latitude city_longitude       date                time  
0          37.3432       -77.4086 2019-12-12 1900-01-01 18:43:00  
1          41.6648       -72.6393 2019-03-22 1900-01-01 18:30:00  
2          45.3814       -75.7085 2019-04-17 1900-01-01 02:00:00  
3          30.6772       -94.0052 2019-04-02 1900-01-01 20:25:00  
4          32.2599       -110.928 2019-05-01 1900-01-01 11:00:00  
...            ...            ...        ...                 ...  
1572       -123.66        48.3486 2011-10-27 1900-01-01 19:30:00  
1573      -87.3963        46.5436 2011-10-27 1900-01-01 21:00:00  
1574      -98.3984        37.0903 2011-08-31 1900-01-01 21:52:00  
1575   

In [94]:
# remove any missing city fields
ufo_final_df

city state  year           date_time     shape     duration  \
0             Chester    VA  2019 2019-12-12 18:43:00     light    5 seconds   
1          Rocky Hill    CT  2019 2019-03-22 18:30:00    circle  3-5 seconds   
2              Ottawa    ON  2019 2019-04-17 02:00:00  teardrop   10 seconds   
3          Kirbyville    TX  2019 2019-04-02 20:25:00      disk   15 minutes   
4              Tucson    AZ  2019 2019-05-01 11:00:00   unknown    5 minutes   
...               ...   ...   ...                 ...       ...          ...   
1572       East Sooke    BC  2011 2011-10-27 19:30:00     light      >5 min.   
1573  Upper Peninsula    MI  2011 2011-10-27 21:00:00     light    5 minutes   
1574         Hazelton    KS  2011 2011-08-31 21:52:00   unknown         2min   
1575  Franklin County    NC  2011 2011-09-01 09:00:00     other        i min   
1576       Wolf Creek    MT  2011 2011-09-01 10:25:00    sphere        short   

                                                  stats  \
0     Occurred : 12/12/2019 18:43  (Entered as : 12/...   
1     Occurred : 3/22/2019 18:30  (Entered as : 03/2...   
2     Occurred : 4/17/2019 02:00  (Entered as : 04-1...   
3     Occurred : 4/2/2019 20:25  (Entered as : 04/02...   
4     Occurred : 5/1/2019 11:00  (Entered as : 5/1/1...   
...                                                 ...   
1572  Occurred : 10/27/2011 19:30  (Entered as : 10/...   
1573  Occurred : 10/27/2011 21:00  (Entered as : 10/...   
1574  Occurred : 8/31/2011 21:52  (Entered as : 08/3...   
1575  Occurred : 9/1/2011 09:00  (Entered as : 09/01...   
1576  Occurred : 9/1/2011 10:25  (Entered as : 09/01...   

                                            report_link  \
0     http://www.nuforc.org/webreports/151/S151739.html   
1     http://www.nuforc.org/webreports/145/S145297.html   
2     http://www.nuforc.org/webreports/145/S145697.html   
3     http://www.nuforc.org/webreports/145/S145476.html   
4     http://www.nuforc.org/webreports/145/S145947.html   
...                                                 ...   
1572   http://www.nuforc.org/webreports/085/S85034.html   
1573   http://www.nuforc.org/webreports/084/S84955.html   
1574   http://www.nuforc.org/webreports/083/S83613.html   
1575   http://www.nuforc.org/webreports/083/S83731.html   
1576   http://www.nuforc.org/webreports/084/S84138.html   

                                                   text               posted  \
0     My wife was driving southeast on a fairly popu...  2019-12-22T00:00:00   
1     I think that I may caught a UFO on the NBC Nig...  2019-03-29T00:00:00   
2     I was driving towards the intersection of fall...  2019-04-18T00:00:00   
3     The object has flashing lights that are green,...  2019-04-08T00:00:00   
4     Description is the same as Washington, DC, eve...  2019-05-09T00:00:00   
...                                                 ...                  ...   
1572  Bright white/blue light moving in fluid manner...  2011-12-12T00:00:00   
1573  Two brilliant red lights, side-by-side, moved ...  2011-12-12T00:00:00   
1574  bright white light in rural Barber county kans...  2011-10-10T00:00:00   
1575  super low altitude stealth jet 9/1/11 NC Extre...  2011-10-10T00:00:00   
1576  Light brown sphere found in the background of ...  2011-10-10T00:00:00   

     city_latitude city_longitude       date                time  
0          37.3432       -77.4086 2019-12-12 1900-01-01 18:43:00  
1          41.6648       -72.6393 2019-03-22 1900-01-01 18:30:00  
2          45.3814       -75.7085 2019-04-17 1900-01-01 02:00:00  
3          30.6772       -94.0052 2019-04-02 1900-01-01 20:25:00  
4          32.2599       -110.928 2019-05-01 1900-01-01 11:00:00  
...            ...            ...        ...                 ...  
1572       -123.66        48.3486 2011-10-27 1900-01-01 19:30:00  
1573      -87.3963        46.5436 2011-10-27 1900-01-01 21:00:00  
1574      -98.3984        37.0903 2011-08-31 1900-01-01 21:52:00  
1575   

In [95]:
#sort by date_time
ufo_final_df = ufo_final_df.sort_values('date_time')
ufo_final_df

city state  year           date_time     shape  \
2930        El Cajon    CA  2010 2010-01-01 00:00:00  triangle   
4198          Benton    AR  2010 2010-01-01 00:00:00    circle   
2932        El Cajon    CA  2010 2010-01-01 00:00:00  triangle   
2931       Park City    KY  2010 2010-01-01 00:00:00     light   
2929     Lemon Grove    CA  2010 2010-01-01 00:00:00     light   
...              ...   ...   ...                 ...       ...   
2760       Strathroy    ON  2019 2019-12-22 20:00:00     light   
2761        Fishkill    NY  2019 2019-12-22 20:35:00      disk   
2762       Bethlehem    PA  2019 2019-12-22 21:00:00   unknown   
2763  Kennett Square    PA  2019 2019-12-22 22:15:00  triangle   
2764       Clarkston    MI  2019 2019-12-22 23:00:00    circle   

              duration                                              stats  \
2930         6 minutes  Occurred : 1/1/2010 00:00  (Entered as : 1/1/2...   
4198           5 mins.  Occurred : 1/1/2010 00:00  (Entered as : 01/01...   
2932        12 minutes  Occurred : 1/1/2010 00:00  (Entered as : 1/1/1...   
2931       2-3 seconds  Occurred : 1/1/2010 00:00  (Entered as : 01/01...   
2929  about 15 minutes  Occurred : 1/1/2010 00:00  (Entered as : 01/01...   
...                ...                                                ...   
2760        15 minutes  Occurred : 12/22/2019 20:00  (Entered as : 12/...   
2761          1 minute  Occurred : 12/22/2019 20:35  (Entered as : 12/...   
2762             Hours  Occurred : 12/22/2019 21:00  (Entered as : 12/...   
2763              3min  Occurred : 12/22/2019 22:15  (Entered as : 12/...   
2764         5 minutes  Occurred : 12/22/2019 23:00  (Entered as : 12/...   

                                            report_link  \
2930   http://www.nuforc.org/webreports/073/S73913.html   
4198   http://www.nuforc.org/webreports/078/S78231.html   
2932   http://www.nuforc.org/webreports/073/S73918.html   
2931   http://www.nuforc.org/webreports/073/S73909.html   
2929   http://www.nuforc.org/webreports/073/S73916.html   
...                                                 ...   
2760  http://www.nuforc.org/webreports/151/S151881.html   
2761  http://www.nuforc.org/webreports/151/S151810.html   
2762  http://www.nuforc.org/webreports/151/S151873.html   
2763  http://www.nuforc.org/webreports/151/S151874.html   
2764  http://www.nuforc.org/webreports/151/S151836.html   

                                                   text               posted  \
2930  On New Years Eve I went outside to hear the ce...  2010-02-14T00:00:00   
4198  4 bright green circles high in the sky going i...  2010-11-21T00:00:00   
2932  3 Red objects hovering over El Cajon CA   Exit...  2010-02-14T00:00:00   
2931  Lights orbiting the moon I am 10,and i was wit...  2010-02-14T00:00:00   
2929  3 Red lights in line pattern above El Cajon/ E...  2010-02-14T00:00:00   
...                                                 ...                  ...   
2760  Chain of white lights following each other upw...  2019-12-22T00:00:00   
2761  Disc Shape Hovering, I was driving on 52 heade...  2019-12-22T00:00:00   
2762  Hello, My wife and I been seeing these objects...  2019-12-22T00:00:00   
2763  Witness to triangle style craft. View was from...  2019-12-22T00:00:00   
2764  7 white lights seen to the west side of the st...  2019-12-22T00:00:00   

     city_latitude city_longitude       date                time  
2930       32.8051       -116.918 2010-01-01 1900-01-01 00:00:00  
4198       34.5759       -92.6351 2010-01-01 1900-01-01 00:00:00  
2932       32.8051       -116.918 2010-01-01 1900-01-01 00:00:00  
2931       37.1336       -86.0986 2010-01-01 1900-01-01 00:00:00  
2929       32.7316       -117.039 2010-01-01 1900-01-01 00:00:00  
...            ...            ...        ...                 ...  
2760       42.9551       -81.6223 2019-12-22 1900-01-01 20:00:00  
2761       41.5277       -73.9072 2019-12-22 1900-01-01 20:35:00  
2762        40.635       

In [97]:
#convert all empty rows into not available
# this step is needed to load into Mongo and route on Flask without any errors 
ufo_final_df = ufo_final_df.replace('','not available', regex=True)
ufo_df

city state  year           date_time     shape     duration  \
0             Chester    VA  2019 2019-12-12 18:43:00     light    5 seconds   
1          Rocky Hill    CT  2019 2019-03-22 18:30:00    circle  3-5 seconds   
2              Ottawa    ON  2019 2019-04-17 02:00:00  teardrop   10 seconds   
3          Kirbyville    TX  2019 2019-04-02 20:25:00      disk   15 minutes   
4              Tucson    AZ  2019 2019-05-01 11:00:00   unknown    5 minutes   
...               ...   ...   ...                 ...       ...          ...   
8825  Franklin County    NC  2011 2011-09-01 09:00:00     other        i min   
8826       Wolf Creek    MT  2011 2011-09-01 10:25:00    sphere        short   
8827          Hamburg    NY  2011 2011-09-01 18:00:00      cone   20 seconds   
8828         Falmouth    MA  2011 2011-09-01 18:45:00     light    3 minutes   
8829           Linton    IN  2011 2011-09-01 20:50:00     light    3 minutes   

                                                  stats  \
0     Occurred : 12/12/2019 18:43  (Entered as : 12/...   
1     Occurred : 3/22/2019 18:30  (Entered as : 03/2...   
2     Occurred : 4/17/2019 02:00  (Entered as : 04-1...   
3     Occurred : 4/2/2019 20:25  (Entered as : 04/02...   
4     Occurred : 5/1/2019 11:00  (Entered as : 5/1/1...   
...                                                 ...   
8825  Occurred : 9/1/2011 09:00  (Entered as : 09/01...   
8826  Occurred : 9/1/2011 10:25  (Entered as : 09/01...   
8827  Occurred : 9/1/2011 18:00  (Entered as : 9/201...   
8828  Occurred : 9/1/2011 18:45  (Entered as : 09/01...   
8829  Occurred : 9/1/2011 20:50  (Entered as : 09/01...   

                                            report_link  \
0     http://www.nuforc.org/webreports/151/S151739.html   
1     http://www.nuforc.org/webreports/145/S145297.html   
2     http://www.nuforc.org/webreports/145/S145697.html   
3     http://www.nuforc.org/webreports/145/S145476.html   
4     http://www.nuforc.org/webreports/145/S145947.html   
...                                                 ...   
8825   http://www.nuforc.org/webreports/083/S83731.html   
8826   http://www.nuforc.org/webreports/084/S84138.html   
8827   http://www.nuforc.org/webreports/084/S84293.html   
8828   http://www.nuforc.org/webreports/083/S83636.html   
8829   http://www.nuforc.org/webreports/083/S83625.html   

                                                   text               posted  \
0     My wife was driving southeast on a fairly popu...  2019-12-22T00:00:00   
1     I think that I may caught a UFO on the NBC Nig...  2019-03-29T00:00:00   
2     I was driving towards the intersection of fall...  2019-04-18T00:00:00   
3     The object has flashing lights that are green,...  2019-04-08T00:00:00   
4     Description is the same as Washington, DC, eve...  2019-05-09T00:00:00   
...                                                 ...                  ...   
8825  super low altitude stealth jet 9/1/11 NC Extre...  2011-10-10T00:00:00   
8826  Light brown sphere found in the background of ...  2011-10-10T00:00:00   
8827  Bun shaped object hovering over parking lot an...  2011-10-10T00:00:00   
8828  Bright silent craft with no navigation lights ...  2011-10-10T00:00:00   
8829  We noticed two bright lights in the dusk sky. ...  2011-10-10T00:00:00   

     city_latitude city_longitude       date                time  
0          37.3432       -77.4086 2019-12-12 1900-01-01 18:43:00  
1          41.6648       -72.6393 2019-03-22 1900-01-01 18:30:00  
2          45.3814       -75.7085 2019-04-17 1900-01-01 02:00:00  
3          30.6772       -94.0052 2019-04-02 1900-01-01 20:25:00  
4          32.2599       -110.928 2019-05-01 1900-01-01 11:00:00  
...            ...            ...        ...                 ...  
8825                              2011-09-01 1900-01-01 09:00:00  
8826                              2011-09-01 1900-01-01 10:25:00  
8827       42.7091       -78.8301 2011-09-01 1900-01-01 18:00:00  
8828   

In [98]:
# save this dataframe as final CSV
ufo_final_df.to_csv("data/UFO_cleaned_2010_2019_v2.csv")

###  Adding 2020 data
- Add in the data that has been scraped by Callum for 2020 only
- Make sure all columns name and the number of columns matches with 2010 -2019 dataframe

In [102]:
#import the csv for 2020
ufo_2020= pd.read_csv('data/ufo2020.csv')
ufo_2020 = pd.DataFrame(ufo_2020)
ufo_2020

Date / Time            City State     Shape   Duration  \
0     12/12/2020 23:37         Hanover    MA       NaN        NaN   
1     12/12/2020 21:01     Star Valley    AZ       NaN        NaN   
2     12/12/2020 21:00        New York    NY     Light  5 minutes   
3     12/12/2020 21:00  West Hempstead    NY     Other  3 minutes   
4     12/12/2020 16:50          Tucson    AZ     Light      5 min   
...                ...             ...   ...       ...        ...   
2853    1/01/2020 0:00       El Centro    CA    Circle  3 minutes   
2854    1/01/2020 0:00           Lacey    WA   Chevron        NaN   
2855    1/01/2020 0:00        Lewiston    ME     Light   On going   
2856         1/01/2020  St. Petersburg    FL  Triangle       3:00   
2857         1/01/2020     San Antonio    TX      Oval  5 minutes   

                                                Summary     Posted  
0                                         MADAR Node 77    1/19/21  
1                                         MADAR Node 53    1/19/21  
2                     BALLS OF LIGHT MOVING IN FIGURE 8   12/23/20  
3     We looked up in the sky and there was a square...   12/23/20  
4     Seen at first as a stationary single point of ...   12/23/20  
...                                                 ...        ...  
2853  Red orb floats slowly changing direction then ...    1/31/20  
2854  Blue light coming from craft, shot through the...    1/31/20  
2855  It left after I came on your site to report it...  2/07/2020  
2856  It's too bad I can't tell the full story on he...  2/07/2020  
2857  looking at fireworks at midnight we observed 1...  2/07/2020  

[2858 rows x 7 columns]

In [105]:
# Mark sure that the column names are labeled the same as our previous dataframe 
#rename the columns to main df
ufo_2020 = ufo_2020.rename(columns={'Date / Time': 'date_time', 'City': 'city', 'State': 'state', 'Shape': 'shape','Duration': 'duration', 'Summary': 'text', 'Posted': 'posted'})
# add missing columns to df
ufo_2020["year"] = 2020
ufo_2020["report_link"] = ""
ufo_2020["city_latitude"] = ""
ufo_2020["city_longitude"] = ""
ufo_2020["stats"] = ""
ufo_2020.head()

date_time            city state  shape   duration  \
0  12/12/2020 23:37         Hanover    MA    NaN        NaN   
1  12/12/2020 21:01     Star Valley    AZ    NaN        NaN   
2  12/12/2020 21:00        New York    NY  Light  5 minutes   
3  12/12/2020 21:00  West Hempstead    NY  Other  3 minutes   
4  12/12/2020 16:50          Tucson    AZ  Light      5 min   

                                                text    posted  year  \
0                                      MADAR Node 77   1/19/21  2020   
1                                      MADAR Node 53   1/19/21  2020   
2                  BALLS OF LIGHT MOVING IN FIGURE 8  12/23/20  2020   
3  We looked up in the sky and there was a square...  12/23/20  2020   
4  Seen at first as a stationary single point of ...  12/23/20  2020   

  report_link city_latitude city_longitude stats  
0                                                 
1                                                 
2                                                 
3                                                 
4

In [108]:
# change the order of columns to match main dataframe
ufo_2020 = ufo_2020[['city', 'state', 'year', 'date_time', 'shape','duration','stats','report_link','text','posted','city_latitude','city_longitude']]
ufo_2020.head()

city state  year         date_time  shape   duration stats  \
0         Hanover    MA  2020  12/12/2020 23:37    NaN        NaN         
1     Star Valley    AZ  2020  12/12/2020 21:01    NaN        NaN         
2        New York    NY  2020  12/12/2020 21:00  Light  5 minutes         
3  West Hempstead    NY  2020  12/12/2020 21:00  Other  3 minutes         
4          Tucson    AZ  2020  12/12/2020 16:50  Light      5 min         

  report_link                                               text    posted  \
0                                                  MADAR Node 77   1/19/21   
1                                                  MADAR Node 53   1/19/21   
2                              BALLS OF LIGHT MOVING IN FIGURE 8  12/23/20   
3              We looked up in the sky and there was a square...  12/23/20   
4              Seen at first as a stationary single point of ...  12/23/20   

  city_latitude city_longitude  
0                               
1                               
2                               
3                               
4

In [117]:
# convert date_time into date time type
# convert to type datetime
ufo_2020['date_time'] = pd.to_datetime(ufo_2020['date_time'])
#ufo_df['date'] = pd.to_datetime(ufo_df['date'], format='%Y-%m-%d')
#ufo_df['time'] = pd.to_datetime(ufo_df['time'], format='%H:%M:%S')
ufo_2020

city state  year           date_time     shape   duration  \
0            Hanover    MA  2020 2020-12-12 23:37:00       NaN        NaN   
1        Star Valley    AZ  2020 2020-12-12 21:01:00       NaN        NaN   
2           New York    NY  2020 2020-12-12 21:00:00     Light  5 minutes   
3     West Hempstead    NY  2020 2020-12-12 21:00:00     Other  3 minutes   
4             Tucson    AZ  2020 2020-12-12 16:50:00     Light      5 min   
...              ...   ...   ...                 ...       ...        ...   
2853       El Centro    CA  2020 2020-01-01 00:00:00    Circle  3 minutes   
2854           Lacey    WA  2020 2020-01-01 00:00:00   Chevron        NaN   
2855        Lewiston    ME  2020 2020-01-01 00:00:00     Light   On going   
2856  St. Petersburg    FL  2020 2020-01-01 00:00:00  Triangle       3:00   
2857     San Antonio    TX  2020 2020-01-01 00:00:00      Oval  5 minutes   

     stats report_link                                               text  \
0                                                           MADAR Node 77   
1                                                           MADAR Node 53   
2                                       BALLS OF LIGHT MOVING IN FIGURE 8   
3                       We looked up in the sky and there was a square...   
4                       Seen at first as a stationary single point of ...   
...    ...         ...                                                ...   
2853                    Red orb floats slowly changing direction then ...   
2854                    Blue light coming from craft, shot through the...   
2855                    It left after I came on your site to report it...   
2856                    It's too bad I can't tell the full story on he...   
2857                    looking at fireworks at midnight we observed 1...   

         posted city_latitude city_longitude  date  time  
0       1/19/21                                NaN   NaN  
1       1/19/21                                NaN   NaN  
2      12/23/20                                NaN   NaN  
3      12/23/20                                NaN   NaN  
4      12/23/20                                NaN   NaN  
...         ...           ...            ...   ...   ...  
2853    1/31/20                                NaN   NaN  
2854    1/31/20                                NaN   NaN  
2855  2/07/2020                                NaN   NaN  
2856  2/07/2020                                NaN   NaN  
2857  2/07/2020                                NaN   NaN  

[2858 rows x 14 columns]

In [125]:
# replace nan with blank
ufo_2020 = ufo_2020.replace('NaN','', regex=True)
ufo_2020

city state  year           date_time     shape   duration  \
0            Hanover    MA  2020 2020-12-12 23:37:00       NaN        NaN   
1        Star Valley    AZ  2020 2020-12-12 21:01:00       NaN        NaN   
2           New York    NY  2020 2020-12-12 21:00:00     Light  5 minutes   
3     West Hempstead    NY  2020 2020-12-12 21:00:00     Other  3 minutes   
4             Tucson    AZ  2020 2020-12-12 16:50:00     Light      5 min   
...              ...   ...   ...                 ...       ...        ...   
2853       El Centro    CA  2020 2020-01-01 00:00:00    Circle  3 minutes   
2854           Lacey    WA  2020 2020-01-01 00:00:00   Chevron        NaN   
2855        Lewiston    ME  2020 2020-01-01 00:00:00     Light   On going   
2856  St. Petersburg    FL  2020 2020-01-01 00:00:00  Triangle       3:00   
2857     San Antonio    TX  2020 2020-01-01 00:00:00      Oval  5 minutes   

     stats report_link                                               text  \
0                                                           MADAR Node 77   
1                                                           MADAR Node 53   
2                                       BALLS OF LIGHT MOVING IN FIGURE 8   
3                       We looked up in the sky and there was a square...   
4                       Seen at first as a stationary single point of ...   
...    ...         ...                                                ...   
2853                    Red orb floats slowly changing direction then ...   
2854                    Blue light coming from craft, shot through the...   
2855                    It left after I came on your site to report it...   
2856                    It's too bad I can't tell the full story on he...   
2857                    looking at fireworks at midnight we observed 1...   

         posted city_latitude city_longitude  date  time  
0       1/19/21                                NaN   NaN  
1       1/19/21                                NaN   NaN  
2      12/23/20                                NaN   NaN  
3      12/23/20                                NaN   NaN  
4      12/23/20                                NaN   NaN  
...         ...           ...            ...   ...   ...  
2853    1/31/20                                NaN   NaN  
2854    1/31/20                                NaN   NaN  
2855  2/07/2020                                NaN   NaN  
2856  2/07/2020                                NaN   NaN  
2857  2/07/2020                                NaN   NaN  

[2858 rows x 14 columns]

## Find missing geo coordinates again
 - since data scrape do not have geo coordinates we have to run our API again
 - Keeping in mind API allows only for 2500 calls per day, we will need multiple API keys

In [126]:
# Set the index to city so that we can do our API search
ufo_2020.set_index("city", inplace = True)
ufo_2020.head()

state  year           date_time  shape   duration stats  \
city                                                                     
Hanover           MA  2020 2020-12-12 23:37:00    NaN        NaN         
Star Valley       AZ  2020 2020-12-12 21:01:00    NaN        NaN         
New York          NY  2020 2020-12-12 21:00:00  Light  5 minutes         
West Hempstead    NY  2020 2020-12-12 21:00:00  Other  3 minutes         
Tucson            AZ  2020 2020-12-12 16:50:00  Light      5 min         

               report_link                                               text  \
city                                                                            
Hanover                                                         MADAR Node 77   
Star Valley                                                     MADAR Node 53   
New York                                    BALLS OF LIGHT MOVING IN FIGURE 8   
West Hempstead              We looked up in the sky and there was a square...   
Tucson                      Seen at first as a stationary single point of ...   

                  posted city_latitude city_longitude  date  time  
city                                                               
Hanover          1/19/21                                NaN   NaN  
Star Valley      1/19/21                                NaN   NaN  
New York        12/23/20                                NaN   NaN  
West Hempstead  12/23/20                                NaN   NaN  
Tucson          12/23/20                                NaN   NaN

In [127]:
# API time
coordinates_20 = []
for line in ufo_2020.index:
    try: 
  #with cities_pd['city'] as f:
    #for line in ufo_loc_pd['city']:
      address = line.strip()
      results = geocoder.geocode(address, no_annotations='1')

      if results and len(results):
        city  = results[0]['formatted']
        longitude = results[0]['geometry']['lng']
        latitude  = results[0]['geometry']['lat']        
        
        #ufo_loc_pd.loc["city","city"] = results[0]['formatted']
        ufo_2020.loc[line,"city_latitude"] = results[0]['geometry']['lng']
        ufo_2020.loc[line,"city_longitude"] = results[0]['geometry']['lat']
    
        response_output = {
            "city": city,
            "city_latitude":latitude,
            "city_longitude":longitude
                    }
        #coordinates.append(response_output)
        coordinates_20.append(response_output)
        
        print(u'%f;%f;%s' % (latitude, longitude, address))
      else:
        sys.stderr.write("not found: %s\n" % address)
        ufo_2020.loc[line,"city_latitude"] = ""
        ufo_2020.loc[line,"city_longitude"] = ""
    
    except (IOError, AttributeError):
        #coordinates = "NaN"
        print('Error: File %s does not appear to exist.' % addressfile)
    except RateLimitExceededError as ex:
      print(ex)
    continue

52.374478;9.738553;Hanover
34.261223;-111.238362;Star Valley
40.712728;-74.006015;New York
40.697580;-73.649673;West Hempstead
32.222876;-110.974848;Tucson
37.888217;-95.317021;Allen
43.850529;-79.020881;Ajax (Canada)
34.185102;-83.925181;Flowery Branch
55.771821;37.715652;rt66
40.851408;-74.044126;Moonachie/Little Ferry
40.910654;-73.562069;Bayville
27.947760;-82.458444;Tampa
44.476160;-73.212906;Burlington
38.254238;-85.759407;Louisville
41.367591;-75.062395;Lords Valley
35.622506;-117.669941;Ridgecrest
48.118146;-123.430741;Port Angeles
33.000023;-80.189793;Summerville
42.774477;1.671794;Aston
33.770859;-116.426874;Rancho mirage
38.871156;-87.441408;Oaktown
41.335933;-71.905904;Stonington
41.875562;-87.624421;Chicago
-37.814218;144.963161;Melbourne
40.599232;-80.472844;Hookstown
34.070958;-84.274733;Alpharetta
40.835934;14.248783;Naples
40.835934;14.248783;Naples
44.083994;-93.226108;Owatonna
38.895037;-77.036543;Washington
39.167718;-119.723438;Carson City
43.126872;0.351334;Lagran

not found: WinstonSalem


41.284222;-81.096486;Garrettsville
33.834752;-117.911732;Anaheim
40.835934;14.248783;Naples
32.222876;-110.974848;Tucson
43.136755;-70.645608;York Harbor
47.531562;-122.638406;Port Orchard
45.716213;-94.048104;Benton
39.755543;-105.221099;Golden
42.609748;-72.597975;Greenfield
34.254208;-110.029833;Show Low
37.639097;-120.996878;Modesto
39.872818;-85.956926;Oaklandon
51.865370;-2.245819;Gloucester
39.955593;-86.013873;Fishers
44.058173;-121.315310;Bend
40.870932;-74.912114;Great Meadows
38.586581;-6.749170;Nogales
39.739236;-104.984862;Denver
39.277616;-74.574600;OCEAN CITY
38.303184;-77.460540;Fredericksburg
42.609748;-72.597975;Greenfield
40.802071;-124.163673;Eureka
47.757320;-122.244015;Kenmore
40.349274;-74.659296;Princeton
37.613825;-122.486919;Pacifica
47.882659;-100.240959;Anamoose
38.833958;-104.825349;Colorado Springs
44.089894;-115.952064;Garden Valley
36.358697;-105.609344;ranchos de taos
49.336457;-0.451253;Juno Beach
28.320155;-80.736647;Rockledge
30.275195;-89.781175;Sli

not found: (Unspecified by witness)


36.766989;-91.915198;Howell
40.846651;-73.878594;New York City (Bronx)
-51.836967;-61.291161;Beaver Island
30.674364;-96.369963;Bryan
38.272231;-90.179248;Monroe
38.652954;-90.241117;St. Louis
30.421309;-87.216915;Pensacola
41.473427;-74.538493;Otisville
36.167256;-115.148516;Las Vegas


...

28.555272;-82.387871;Brooksville
31.059683;-7.900009;Atlas
25.855374;-80.193103;Portal
43.143307;-91.932664;Fort Atkinson
40.262570;-80.187280;Canonsburg
40.349274;-74.659296;Princeton
42.250594;-83.178536;Lincoln Park
39.799017;-89.643957;Springfield
39.616112;-106.717284;Eagle
35.199800;-87.030840;Bonduel/Pulaski
44.760644;-85.616530;Traverse City
42.594105;-86.104596;Fennville
43.420039;-83.949037;Saginaw
39.063956;-108.550732;Grand Junction
43.331841;-110.776892;Hoback
41.258746;-95.938376;Omaha
40.297182;-111.694943;Orem
41.573864;-97.082849;Colfax
39.028609;-96.831398;Junction City
43.190039;-112.348357;Blackfoot
43.074761;-89.383761;Madison
51.456659;-0.969651;Reading
43.074761;-89.383761;Madison
38.883886;-94.818870;Olathe
50.371266;-4.142566;Plymouth
39.028609;-96.831398;Junction City
42.041582;-87.887392;Des Plaines
53.479489;-2.245115;Manchester
41.975887;-91.670405;Cedar Rapids
33.157058;-94.969401;Mount Pleasant
39.800550;-105.081157;Arvada
38.723659;-89.955935;Maryville
4

not found: Frutia


40.346220;-74.287924;Robertsville
36.941812;-120.171438;Madera
36.062584;-94.157433;Fayetteville
52.135016;-0.012388;Orwell
28.299618;-82.452270;Pasco
49.033333;-119.466667;Osoyoos (Canada)
39.017835;-85.058843;Dillsboro
53.136241;-109.628509;Lashburn (Canada)
53.190887;-2.890896;Chester
46.050293;-71.981457;Ile Perrot (Canada)
37.542094;-98.122172;Kingman
38.933431;-74.921423;Cape May
39.364285;-74.422935;Atlantic City
40.441694;-79.990086;Pittsburgh
40.550853;-105.066808;Fort Collins
38.588939;-89.990382;Fairview Heights
28.364725;-82.195918;Dade City
36.739442;-119.784831;Fresno
47.507594;6.480773;Fallon
40.441694;-79.990086;Pittsburg
38.356577;-121.987744;Vacaville
37.739651;-121.425223;Tracy
48.742314;100.006187;Galt
43.767797;-94.019682;Delavan
37.957702;-121.290780;Stockton
27.965853;-82.800103;Clearwater
38.552621;-78.913361;Singers Glen
47.549025;1.732406;La Center
39.742759;-104.990988;Central Colorado
45.520247;-122.674195;Portland


not found: Fairplat


43.389663;-123.312360;Sutherlin
42.322260;-83.176315;Dearborn
35.489417;-120.670726;Atascadero
37.804456;-122.271356;Oakland
37.463552;-122.428586;Half Moon Bay
45.299840;-122.773710;Canby/Wilsonville
41.307135;-111.960199;North Ogden
47.187019;-122.170229;Bonney Lake
39.976388;-95.296922;White Cloud
41.975887;-91.670405;Cedar Rapids
50.536715;2.408654;Ames
45.074854;-93.329630;Brooklyn Center
43.157285;-77.615214;Rochester
40.038130;-76.305669;Lancaster
38.037297;-84.896617;Lawrenceburg
41.146152;-73.943295;Congers
37.690483;-122.472670;Daly City
41.065924;-75.319906;Scotrun
36.115631;-97.058572;Stillwater
44.026200;-90.073713;Necedah
52.366681;8.313543;Bohmte (Germany)
33.306160;-111.841250;Chandler
42.765443;-71.439717;Hudson
36.206447;-86.832887;Nashville (Whites Creek)
38.531463;-84.036872;Mount Olivet
37.692236;-97.337545;Wichita
54.894848;-2.936231;Carlisle
44.820596;-73.507081;West Chazy
37.030097;-76.345206;Hampton
37.270973;-79.941431;Roanoke
41.897515;-84.037305;Adrian
46.43

## Multiple API Calls
- Since Open Cage API only allows 2500 API calls and our data has 2853 rows
- We need to import another API key and run again for missin fields

In [135]:
# API for missing lat lng 2nd run
ufo_2020_api2 = ufo_2020.loc[ufo_2020['city_latitude'] == ""]
ufo_2020_api2 = pd.DataFrame(ufo_2020_api2)
ufo_2020_api2

state  year           date_time     shape  \
city                                                                  
WinstonSalem                 NC  2020 2020-12-10 03:15:00     Other   
(Unspecified by witness)     KY  2020 2020-10-10 21:00:00     Light   
bidenhospice                 AK  2020 2020-10-10 14:00:00   Chevron   
Teerre Haute                 IN  2020 2020-10-06 19:35:00     Cigar   
NaN                         NaN  2020 2020-09-11 00:00:00       NaN   
...                         ...   ...                 ...       ...   
Portimao Alvor (Portugal)   NaN  2020 2020-01-01 00:15:00     Other   
Round Rock                   TX  2020 2020-01-01 00:12:00     Light   
San Marcos                   TX  2020 2020-01-01 00:03:00    Sphere   
El Centro                    CA  2020 2020-01-01 00:00:00    Circle   
St. Petersburg               FL  2020 2020-01-01 00:00:00  Triangle   

                               duration stats report_link  \
city                                                        
WinstonSalem               45 mins plus                     
(Unspecified by witness)         1 hour                     
bidenhospice                         45                     
Teerre Haute                 10 minutes                     
NaN                                 NaN                     
...                                 ...   ...         ...   
Portimao Alvor (Portugal)     8 minutes                     
Round Rock                   10 minutes                     
San Marcos                   10 minutes                     
El Centro                     3 minutes                     
St. Petersburg                     3:00                     

                                                                        text  \
city                                                                           
WinstonSalem               A bright not moving object with pulsating red ...   
(Unspecified by witness)                       Lights moving around the sky.   
bidenhospice               Why in the hell have you people just stopped u...   
Teerre Haute               This unknown orange glowing object became appa...   
NaN                        I am a 19 year corrections officer. During my ...   
...                                                                      ...   
Portimao Alvor (Portugal)  Dozens of stars traveling east to North West, ...   
Round Rock                 Looking out bedroom window towards the west. W...   
San Marcos                 Blue Orb Sighting &quot;video recording&quot; ...   
El Centro                  Red orb floats slowly changing direction then ...   
St. Petersburg             It's too bad I can't tell the full story on he...   

                               posted city_latitude city_longitude  date  time  
city                                                                            
WinstonSalem                 12/23/20                                NaN   NaN  
(Unspecified by witness)     12/23/20                                NaN   NaN  
bidenhospice                 12/23/20                                NaN   NaN  
Teerre Haute                 12/23/20                                NaN   NaN  
NaN                        11/05/2020                                NaN   NaN  
...                               ...           ...            ...   ...   ...  
Portimao Alvor (Portugal)   2/07/2020                                NaN   NaN  
Round Rock                  2/07/2020                                NaN   NaN  
San Marcos                    1/19/21                                NaN   NaN  
El Centro                     1/31/20                                NaN   NaN  
St. Petersburg              2/07/2020                                NaN   NaN  

[250 rows x 13 columns]

In [137]:
# API time
coordinates_20_2 = []
for line in ufo_2020_api2.index:
    try: 
  #with cities_pd['city'] as f:
    #for line in ufo_loc_pd['city']:
      address = line.strip()
      results = geocoder.geocode(address, no_annotations='1')

      if results and len(results):
        city  = results[0]['formatted']
        longitude = results[0]['geometry']['lng']
        latitude  = results[0]['geometry']['lat']        
        
        #ufo_loc_pd.loc["city","city"] = results[0]['formatted']
        ufo_2020_api2.loc[line,"city_latitude"] = results[0]['geometry']['lng']
        ufo_2020_api2.loc[line,"city_longitude"] = results[0]['geometry']['lat']
    
        response_output = {
            "city": city,
            "city_latitude":latitude,
            "city_longitude":longitude
                    }
        #coordinates.append(response_output)
        coordinates_20_2.append(response_output)
        
        print(u'%f;%f;%s' % (latitude, longitude, address))
      else:
        sys.stderr.write("not found: %s\n" % address)
        ufo_2020_api2.loc[line,"city_latitude"] = ""
        ufo_2020_api2.loc[line,"city_longitude"] = ""
    
    except (IOError, AttributeError):
        #coordinates = "NaN"
        print('Error: File %s does not appear to exist.' % addressfile)
    except RateLimitExceededError as ex:
      print(ex)
    continue

not found: WinstonSalem
not found: (Unspecified by witness)
not found: bidenhospice
not found: Teerre Haute


Error: File       index                         city state  year           date_time  \
0         0  Caloocan City (Philippines)        2019 2019-06-06 19:00:00   
1         1                       Leyner    CO  2019 2019-08-09 20:22:00   
2         2                     Catalina    AZ  2019 2019-08-09 20:30:00   
3         3                 Marin County    CA  2019 2019-08-01 22:30:00   
4         4             West Bowmanstown    PA  2019 2019-06-27 01:00:00   
...     ...                          ...   ...   ...                 ...   
1572   1572                   East Sooke    BC  2011 2011-10-27 19:30:00   
1573   1573              Upper Peninsula    MI  2011 2011-10-27 21:00:00   
1574   1574                     Hazelton    KS  2011 2011-08-31 21:52:00   
1575   1575              Franklin County    NC  2011 2011-09-01 09:00:00   
1576   1576                   Wolf Creek    MT  2011 2011-09-01 10:25:00   

         shape                   duration  \
0        other             19:

not found: Lake Kachess (Easbound on I-90)


Error: File       index                         city state  year           date_time  \
0         0  Caloocan City (Philippines)        2019 2019-06-06 19:00:00   
1         1                       Leyner    CO  2019 2019-08-09 20:22:00   
2         2                     Catalina    AZ  2019 2019-08-09 20:30:00   
3         3                 Marin County    CA  2019 2019-08-01 22:30:00   
4         4             West Bowmanstown    PA  2019 2019-06-27 01:00:00   
...     ...                          ...   ...   ...                 ...   
1572   1572                   East Sooke    BC  2011 2011-10-27 19:30:00   
1573   1573              Upper Peninsula    MI  2011 2011-10-27 21:00:00   
1574   1574                     Hazelton    KS  2011 2011-08-31 21:52:00   
1575   1575              Franklin County    NC  2011 2011-09-01 09:00:00   
1576   1576                   Wolf Creek    MT  2011 2011-09-01 10:25:00   

         shape                   duration  \
0        other             19:

not found: Aidrie (Canada)
not found: Sandord
not found: Scapoose
not found: Berkeleysprings
not found: Sunnappee
not found: Cuthougue


...

41.661164;-86.180703;Mishawaka
44.807118;-93.787179;Carver
30.694357;-88.043054;Mobile
37.452250;-93.233113;Goodnight
33.605937;-86.021359;Eastaboga
39.702174;44.298798;Ararat
35.846040;-86.392110;Murfreesboro
25.091338;-80.467774;Key Largo
51.187463;-114.471071;Cochrane (Canada)
50.117190;-122.954302;Whistler (Canada)
40.861730;-80.286451;Ellwood city
29.989282;-97.877174;Kyle
31.187132;-94.781041;Diboll
35.214632;-109.478668;Apache
39.995573;-82.674237;Pataskala
38.418575;-90.487134;Shady Valley
38.525049;-89.133404;Centralia
51.487517;-0.168701;Chelsea
40.758390;-82.515447;Mansfield
47.657110;-117.261394;Spokane Valley
36.167256;-115.148516;Las vegas
34.448050;-119.242889;Ojai
33.394436;-86.886622;Ross Bridge
33.458103;-86.952333;Hueytown
42.113679;-86.050156;Hamilton Township
-35.257624;138.895680;Strathalbyn (Australia)
-25.326389;31.014444;White River (South Africa)
41.075469;-73.775130;Valhalla
33.772179;-116.495298;Palm Springs
45.522894;-122.989827;Hillsboro
38.870842;-94.1738

not found: Progrresso (Yucatan)(Mexico)


40.886016;-74.007257;Teaneck
42.638980;-71.382840;North Chelmsford
34.055033;-117.182744;Redlands
29.894695;-81.314539;St Augustine
26.271192;-80.270604;Coral Springs
40.760789;-72.835420;Mastic Beach
27.197983;-80.251918;Stuart
27.855760;-80.453501;Sebastian Inlet
13.581921;-38.320312;Atlantic Ocean
22.253999;84.857752;Rourkela (India)
18.335653;-64.956052;St. Thomas (USVI)
44.731909;-93.217720;Apple Valley
26.640628;-81.872308;M Fort Myers
-38.144473;145.126881;frankston melbourne
31.986994;-81.149027;Alden Williams
34.053691;-118.242766;Los angeles
39.526121;-119.812658;Reno
26.609151;-81.652407;Lehigh Acres
8.773424;-82.637576;Volcan (Republic of Panama)
43.716241;-96.501138;Garretson
30.239687;-81.385638;Ponte Vedra Beach
40.366432;-105.502855;Estes Park
42.451302;-73.260018;Pittsfield
41.063951;-80.056447;Slippery Rock
44.817813;20.456897;Belgrade
30.047469;-99.140021;Kerrville
39.978802;-86.118926;Carmel
37.981158;-88.329210;Norris City
18.035255;-66.848705;Yauco (Puerto Rico)
4

not found: Shelby Townshiip


50.097679;-104.725800;Wilcox (near)
41.079990;-85.138601;Fort Wayne
35.814431;-94.628616;Stilwell
51.647330;-0.131255;Oakwood
34.150872;-118.448987;Sherman Oaks
38.839894;-83.505170;Ada
45.383855;-122.766352;Tualatin
35.143922;-103.418343;Quay
50.412229;-3.575560;Whiterock
33.677324;-86.396648;Odenville
50.678797;-120.329794;Kamloops (Kamloops)
32.735729;-101.955020;Lamesa
33.320214;-2.423847;Oriental
33.751497;-84.747714;Douglasville
39.928446;-75.273796;Upper Darby
32.838383;-116.973917;Santee
51.898627;-8.470594;Cork (Ireland)
31.452276;-83.509687;Tifton
34.067617;-118.087976;Rosemead
44.408178;-64.956188;North Brookfield
41.007152;-72.554137;Mattituck
60.330143;-151.236631;Kasilof
46.828071;-124.085752;Grayland
33.748992;-84.390264;Atlanta
28.218899;-82.457594;Land o' Lakes
39.764002;-75.553812;Augustine (at)
26.452025;-81.948145;Fort Myers Beach
39.768396;-83.057130;Commercial Point
51.046095;-114.065465;Calgary (Canada)
31.815142;-101.045267;Sterling
43.661554;-97.794536;Hanson
4

In [140]:
# filter out anything that we could not get data for
ufo_2020_api2 = ufo_2020_api2.loc[ufo_2020_api2['city_latitude'] != ""]
ufo_2020_api2 = pd.DataFrame(ufo_2020_api2)
ufo_2020_api2

state  year           date_time     shape  \
city                                                                  
Comstock Park                MI  2020 2020-02-07 01:30:00     Light   
Wooster                      OH  2020 2020-02-06 21:40:00     Flash   
Socorro (south of)           NM  2020 2020-02-06 18:34:00  Changing   
Mobile                       AL  2020 2020-02-06 18:26:00       NaN   
The Dalles                   OR  2020 2020-02-06 17:30:00      Disk   
...                         ...   ...                 ...       ...   
Portimao Alvor (Portugal)   NaN  2020 2020-01-01 00:15:00     Other   
Round Rock                   TX  2020 2020-01-01 00:12:00     Light   
San Marcos                   TX  2020 2020-01-01 00:03:00    Sphere   
El Centro                    CA  2020 2020-01-01 00:00:00    Circle   
St. Petersburg               FL  2020 2020-01-01 00:00:00  Triangle   

                             duration stats report_link  \
city                                                      
Comstock Park                       2                     
Wooster                      1 second                     
Socorro (south of)         25 minutes                     
Mobile                            NaN                     
The Dalles                 10 seconds                     
...                               ...   ...         ...   
Portimao Alvor (Portugal)   8 minutes                     
Round Rock                 10 minutes                     
San Marcos                 10 minutes                     
El Centro                   3 minutes                     
St. Petersburg                   3:00                     

                                                                        text  \
city                                                                           
Comstock Park              Around 12 am is when i first noticed outside o...   
Wooster                    There was a bright turquoise flash in the sky,...   
Socorro (south of)         On the evening of February 6, 2020 around appr...   
Mobile                                                         MADAR Node 63   
The Dalles                          Large disk above town moving slowly away   
...                                                                      ...   
Portimao Alvor (Portugal)  Dozens of stars traveling east to North West, ...   
Round Rock                 Looking out bedroom window towards the west. W...   
San Marcos                 Blue Orb Sighting &quot;video recording&quot; ...   
El Centro                  Red orb floats slowly changing direction then ...   
St. Petersburg             It's too bad I can't tell the full story on he...   

                              posted city_latitude city_longitude  date  time  
city                                                                           
Comstock Park              2/07/2020      -85.6655        43.0343   NaN   NaN  
Wooster                    2/07/2020      -81.9398        40.7981   NaN   NaN  
Socorro (south of)           2/13/20      -106.893        34.0573   NaN   NaN  
Mobile                       2/25/20      -88.0431        30.6944   NaN   NaN  
The Dalles                   2/25/20      -121.184        45.6015   NaN   NaN  
...                              ...           ...            ...   ...   ...  
Portimao Alvor (Portugal)  2/07/2020      -8.56302        37.1508   NaN   NaN  
Round Rock                 2/07/2020      -97.6789        30.5082   NaN   NaN  
San Marcos                   1/19/21      -97.9406        29.8826   NaN   NaN  
El Centro                    1/31/20      -115.563         32.792   NaN   NaN  
St. Petersburg             2/07/2020       30.3162        59.9387   NaN   NaN  

[224 rows x 13 columns]

In [148]:
# Combine all 2020 data
ufo_2020_final = pd.concat([ufo_2020_1,ufo_2020_api2], axis=0)
ufo_2020_final

state  year           date_time     shape  \
city                                                                  
Hanover                      MA  2020 2020-12-12 23:37:00       NaN   
Star Valley                  AZ  2020 2020-12-12 21:01:00       NaN   
New York                     NY  2020 2020-12-12 21:00:00     Light   
West Hempstead               NY  2020 2020-12-12 21:00:00     Other   
Tucson                       AZ  2020 2020-12-12 16:50:00     Light   
...                         ...   ...                 ...       ...   
Portimao Alvor (Portugal)   NaN  2020 2020-01-01 00:15:00     Other   
Round Rock                   TX  2020 2020-01-01 00:12:00     Light   
San Marcos                   TX  2020 2020-01-01 00:03:00    Sphere   
El Centro                    CA  2020 2020-01-01 00:00:00    Circle   
St. Petersburg               FL  2020 2020-01-01 00:00:00  Triangle   

                             duration stats report_link  \
city                                                      
Hanover                           NaN                     
Star Valley                       NaN                     
New York                    5 minutes                     
West Hempstead              3 minutes                     
Tucson                          5 min                     
...                               ...   ...         ...   
Portimao Alvor (Portugal)   8 minutes                     
Round Rock                 10 minutes                     
San Marcos                 10 minutes                     
El Centro                   3 minutes                     
St. Petersburg                   3:00                     

                                                                        text  \
city                                                                           
Hanover                                                        MADAR Node 77   
Star Valley                                                    MADAR Node 53   
New York                                   BALLS OF LIGHT MOVING IN FIGURE 8   
West Hempstead             We looked up in the sky and there was a square...   
Tucson                     Seen at first as a stationary single point of ...   
...                                                                      ...   
Portimao Alvor (Portugal)  Dozens of stars traveling east to North West, ...   
Round Rock                 Looking out bedroom window towards the west. W...   
San Marcos                 Blue Orb Sighting &quot;video recording&quot; ...   
El Centro                  Red orb floats slowly changing direction then ...   
St. Petersburg             It's too bad I can't tell the full story on he...   

                              posted city_latitude city_longitude  date  time  
city                                                                           
Hanover                      1/19/21       9.73855        52.3745   NaN   NaN  
Star Valley                  1/19/21      -111.238        34.2612   NaN   NaN  
New York                    12/23/20       -74.006        40.7127   NaN   NaN  
West Hempstead              12/23/20      -73.6497        40.6976   NaN   NaN  
Tucson                      12/23/20      -110.975        32.2229   NaN   NaN  
...                              ...           ...            ...   ...   ...  
Portimao Alvor (Portugal)  2/07/2020      -8.56302        37.1508   NaN   NaN  
Round Rock                 2/07/2020      -97.6789        30.5082   NaN   NaN  
San Marcos                   1/19/21      -97.9406        29.8826   NaN   NaN  
El Centro                    1/31/20      -115.563         32.792   NaN   NaN  
St. Petersburg             2/07/2020       30.3162        59.9387   NaN   NaN  

[2832 rows x 13 columns]

In [149]:
# reset index of df
ufo_2020_final.reset_index(inplace=True)

In [157]:
ufo_2020_final = pd.DataFrame(ufo_2020_final)

In [171]:
# replace nan with blank
ufo_2020_final = ufo_2020_final.replace('NaN','not available', regex=True)
ufo_2020_final = ufo_2020_final.replace('','not available', regex=True)
ufo_2020_final

city state  year           date_time     shape  \
0                       Hanover    MA  2020 2020-12-12 23:37:00       NaN   
1                   Star Valley    AZ  2020 2020-12-12 21:01:00       NaN   
2                      New York    NY  2020 2020-12-12 21:00:00     Light   
3                West Hempstead    NY  2020 2020-12-12 21:00:00     Other   
4                        Tucson    AZ  2020 2020-12-12 16:50:00     Light   
...                         ...   ...   ...                 ...       ...   
2827  Portimao Alvor (Portugal)   NaN  2020 2020-01-01 00:15:00     Other   
2828                 Round Rock    TX  2020 2020-01-01 00:12:00     Light   
2829                 San Marcos    TX  2020 2020-01-01 00:03:00    Sphere   
2830                  El Centro    CA  2020 2020-01-01 00:00:00    Circle   
2831             St. Petersburg    FL  2020 2020-01-01 00:00:00  Triangle   

        duration          stats    report_link  \
0            NaN  not available  not available   
1            NaN  not available  not available   
2      5 minutes  not available  not available   
3      3 minutes  not available  not available   
4          5 min  not available  not available   
...          ...            ...            ...   
2827   8 minutes  not available  not available   
2828  10 minutes  not available  not available   
2829  10 minutes  not available  not available   
2830   3 minutes  not available  not available   
2831        3:00  not available  not available   

                                                   text     posted  \
0                                         MADAR Node 77    1/19/21   
1                                         MADAR Node 53    1/19/21   
2                     BALLS OF LIGHT MOVING IN FIGURE 8   12/23/20   
3     We looked up in the sky and there was a square...   12/23/20   
4     Seen at first as a stationary single point of ...   12/23/20   
...                                                 ...        ...   
2827  Dozens of stars traveling east to North West, ...  2/07/2020   
2828  Looking out bedroom window towards the west. W...  2/07/2020   
2829  Blue Orb Sighting &quot;video recording&quot; ...    1/19/21   
2830  Red orb floats slowly changing direction then ...    1/31/20   
2831  It's too bad I can't tell the full story on he...  2/07/2020   

      city_latitude  city_longitude  date  time  
0          9.738553       52.374478   NaN   NaN  
1       -111.238362       34.261223   NaN   NaN  
2        -74.006015       40.712728   NaN   NaN  
3        -73.649673       40.697580   NaN   NaN  
4       -110.974848       32.222876   NaN   NaN  
...             ...             ...   ...   ...  
2827      -8.563022       37.150762   NaN   NaN  
2828     -97.678893       30.508235   NaN   NaN  
2829     -97.940583       29.882644   NaN   NaN  
2830    -115.563051       32.792000   NaN   NaN  
2831      30.316229       59.938732   NaN   NaN  

[2832 rows x 14 columns]

## Combine all dataframe for export and load
- We now combine 2010 - 2019 data with what we've just created for 2020 for a complete dataset
- With only useable data

In [172]:
# Combine all previous year data with 2020 data for complete dataset
ufo_complete = pd.concat([ufo_final_df,ufo_2020_final], axis=0)
ufo_complete.head()

city state  year  date_time     shape          duration  \
2930     El Cajon    CA  2010 2010-01-01  triangle         6 minutes   
4198       Benton    AR  2010 2010-01-01    circle           5 mins.   
2932     El Cajon    CA  2010 2010-01-01  triangle        12 minutes   
2931    Park City    KY  2010 2010-01-01     light       2-3 seconds   
2929  Lemon Grove    CA  2010 2010-01-01     light  about 15 minutes   

                                                  stats  \
2930  Occurred : 1/1/2010 00:00  (Entered as : 1/1/2...   
4198  Occurred : 1/1/2010 00:00  (Entered as : 01/01...   
2932  Occurred : 1/1/2010 00:00  (Entered as : 1/1/1...   
2931  Occurred : 1/1/2010 00:00  (Entered as : 01/01...   
2929  Occurred : 1/1/2010 00:00  (Entered as : 01/01...   

                                           report_link  \
2930  http://www.nuforc.org/webreports/073/S73913.html   
4198  http://www.nuforc.org/webreports/078/S78231.html   
2932  http://www.nuforc.org/webreports/073/S73918.html   
2931  http://www.nuforc.org/webreports/073/S73909.html   
2929  http://www.nuforc.org/webreports/073/S73916.html   

                                                   text               posted  \
2930  On New Years Eve I went outside to hear the ce...  2010-02-14T00:00:00   
4198  4 bright green circles high in the sky going i...  2010-11-21T00:00:00   
2932  3 Red objects hovering over El Cajon CA   Exit...  2010-02-14T00:00:00   
2931  Lights orbiting the moon I am 10,and i was wit...  2010-02-14T00:00:00   
2929  3 Red lights in line pattern above El Cajon/ E...  2010-02-14T00:00:00   

     city_latitude city_longitude       date       time  
2930       32.8051       -116.918 2010-01-01 1900-01-01  
4198       34.5759       -92.6351 2010-01-01 1900-01-01  
2932       32.8051       -116.918 2010-01-01 1900-01-01  
2931       37.1336       -86.0986 2010-01-01 1900-01-01  
2929       32.7316       -117.039 2010-01-01 1900-01-01

In [173]:
ufo_complete = pd.DataFrame(ufo_complete)
ufo_complete

city state  year           date_time     shape  \
2930                   El Cajon    CA  2010 2010-01-01 00:00:00  triangle   
4198                     Benton    AR  2010 2010-01-01 00:00:00    circle   
2932                   El Cajon    CA  2010 2010-01-01 00:00:00  triangle   
2931                  Park City    KY  2010 2010-01-01 00:00:00     light   
2929                Lemon Grove    CA  2010 2010-01-01 00:00:00     light   
...                         ...   ...   ...                 ...       ...   
2827  Portimao Alvor (Portugal)   NaN  2020 2020-01-01 00:15:00     Other   
2828                 Round Rock    TX  2020 2020-01-01 00:12:00     Light   
2829                 San Marcos    TX  2020 2020-01-01 00:03:00    Sphere   
2830                  El Centro    CA  2020 2020-01-01 00:00:00    Circle   
2831             St. Petersburg    FL  2020 2020-01-01 00:00:00  Triangle   

              duration                                              stats  \
2930         6 minutes  Occurred : 1/1/2010 00:00  (Entered as : 1/1/2...   
4198           5 mins.  Occurred : 1/1/2010 00:00  (Entered as : 01/01...   
2932        12 minutes  Occurred : 1/1/2010 00:00  (Entered as : 1/1/1...   
2931       2-3 seconds  Occurred : 1/1/2010 00:00  (Entered as : 01/01...   
2929  about 15 minutes  Occurred : 1/1/2010 00:00  (Entered as : 01/01...   
...                ...                                                ...   
2827         8 minutes                                      not available   
2828        10 minutes                                      not available   
2829        10 minutes                                      not available   
2830         3 minutes                                      not available   
2831              3:00                                      not available   

                                           report_link  \
2930  http://www.nuforc.org/webreports/073/S73913.html   
4198  http://www.nuforc.org/webreports/078/S78231.html   
2932  http://www.nuforc.org/webreports/073/S73918.html   
2931  http://www.nuforc.org/webreports/073/S73909.html   
2929  http://www.nuforc.org/webreports/073/S73916.html   
...                                                ...   
2827                                     not available   
2828                                     not available   
2829                                     not available   
2830                                     not available   
2831                                     not available   

                                                   text               posted  \
2930  On New Years Eve I went outside to hear the ce...  2010-02-14T00:00:00   
4198  4 bright green circles high in the sky going i...  2010-11-21T00:00:00   
2932  3 Red objects hovering over El Cajon CA   Exit...  2010-02-14T00:00:00   
2931  Lights orbiting the moon I am 10,and i was wit...  2010-02-14T00:00:00   
2929  3 Red lights in line pattern above El Cajon/ E...  2010-02-14T00:00:00   
...                                                 ...                  ...   
2827  Dozens of stars traveling east to North West, ...            2/07/2020   
2828  Looking out bedroom window towards the west. W...            2/07/2020   
2829  Blue Orb Sighting &quot;video recording&quot; ...              1/19/21   
2830  Red orb floats slowly changing direction then ...              1/31/20   
2831  It's too bad I can't tell the full story on he...            2/07/2020   

     city_latitude city_longitude       date       time  
2930       32.8051       -116.918 2010-01-01 1900-01-01  
4198       34.5759       -92.6351 2010-01-01 1900-01-01  
2932       32.8051       -116.918 2010-01-01 1900-01-01  
2931       37.1336       -86.0986 2010-01-01 1900-01-01  
2929       32.7316       -117.039 2010-01-01 1900-01-01  
...            ...            ...        ...        ...  
2827      -8.56302        37.1508        NaT        NaT  
2828      -97.6789        30.5082        NaT        NaT

## Data Load
- Convert into CSV format and give to Renzo to load into MongoDB

In [166]:
# Save to CSV
ufo_complete.to_csv("data/UFO_data_2010_2020.csv")

## Sencond Check of Data to remove all empty fields

In [3]:
# Dataset
ufo_data2020 = pd.read_csv('data/UFO_data_2010_2020.csv')
ufo_data2020.head()

city state  year       date_time     shape          duration  \
0     El Cajon    CA  2010  1/01/2010 0:00  triangle         6 minutes   
1       Benton    AR  2010  1/01/2010 0:00    circle           5 mins.   
2     El Cajon    CA  2010  1/01/2010 0:00  triangle        12 minutes   
3    Park City    KY  2010  1/01/2010 0:00     light       2-3 seconds   
4  Lemon Grove    CA  2010  1/01/2010 0:00     light  about 15 minutes   

                                               stats  \
0  Occurred : 1/1/2010 00:00  (Entered as : 1/1/2...   
1  Occurred : 1/1/2010 00:00  (Entered as : 01/01...   
2  Occurred : 1/1/2010 00:00  (Entered as : 1/1/1...   
3  Occurred : 1/1/2010 00:00  (Entered as : 01/01...   
4  Occurred : 1/1/2010 00:00  (Entered as : 01/01...   

                                        report_link  \
0  http://www.nuforc.org/webreports/073/S73913.html   
1  http://www.nuforc.org/webreports/078/S78231.html   
2  http://www.nuforc.org/webreports/073/S73918.html   
3  http://www.nuforc.org/webreports/073/S73909.html   
4  http://www.nuforc.org/webreports/073/S73916.html   

                                                text               posted  \
0  On New Years Eve I went outside to hear the ce...  2010-02-14T00:00:00   
1  4 bright green circles high in the sky going i...  2010-11-21T00:00:00   
2  3 Red objects hovering over El Cajon CA   Exit...  2010-02-14T00:00:00   
3  Lights orbiting the moon I am 10,and i was wit...  2010-02-14T00:00:00   
4  3 Red lights in line pattern above El Cajon/ E...  2010-02-14T00:00:00   

  city_latitude city_longitude       date     time  
0   32.80505053   -116.9175684  1/01/2010  0:00:00  
1   34.57594588   -92.63512706  1/01/2010  0:00:00  
2   32.80505053   -116.9175684  1/01/2010  0:00:00  
3       37.1336       -86.0986  1/01/2010  0:00:00  
4   32.73158974   -117.0389026  1/01/2010  0:00:00

In [4]:
#convert all empty rows into not available
ufo_data2020 = ufo_data2020.replace('','not available', regex=True)
ufo_data2020

city          state  year       date_time  \
0                       El Cajon             CA  2010  1/01/2010 0:00   
1                         Benton             AR  2010  1/01/2010 0:00   
2                       El Cajon             CA  2010  1/01/2010 0:00   
3                      Park City             KY  2010  1/01/2010 0:00   
4                    Lemon Grove             CA  2010  1/01/2010 0:00   
...                          ...            ...   ...             ...   
11656  Portimao Alvor (Portugal)  not available  2020  1/01/2020 0:15   
11657                 Round Rock             TX  2020  1/01/2020 0:12   
11658                 San Marcos             TX  2020  1/01/2020 0:03   
11659                  El Centro             CA  2020  1/01/2020 0:00   
11660             St. Petersburg             FL  2020  1/01/2020 0:00   

          shape          duration  \
0      triangle         6 minutes   
1        circle           5 mins.   
2      triangle        12 minutes   
3         light       2-3 seconds   
4         light  about 15 minutes   
...         ...               ...   
11656     Other         8 minutes   
11657     Light        10 minutes   
11658    Sphere        10 minutes   
11659    Circle         3 minutes   
11660  Triangle              3:00   

                                                   stats  \
0      Occurred : 1/1/2010 00:00  (Entered as : 1/1/2...   
1      Occurred : 1/1/2010 00:00  (Entered as : 01/01...   
2      Occurred : 1/1/2010 00:00  (Entered as : 1/1/1...   
3      Occurred : 1/1/2010 00:00  (Entered as : 01/01...   
4      Occurred : 1/1/2010 00:00  (Entered as : 01/01...   
...                                                  ...   
11656                                      not available   
11657                                      not available   
11658                                      not available   
11659                                      not available   
11660                                      not available   

                                            report_link  \
0      http://www.nuforc.org/webreports/073/S73913.html   
1      http://www.nuforc.org/webreports/078/S78231.html   
2      http://www.nuforc.org/webreports/073/S73918.html   
3      http://www.nuforc.org/webreports/073/S73909.html   
4      http://www.nuforc.org/webreports/073/S73916.html   
...                                                 ...   
11656                                     not available   
11657                                     not available   
11658                                     not available   
11659                                     not available   
11660                                     not available   

                                                    text               posted  \
0      On New Years Eve I went outside to hear the ce...  2010-02-14T00:00:00   
1      4 bright green circles high in the sky going i...  2010-11-21T00:00:00   
2      3 Red objects hovering over El Cajon CA   Exit...  2010-02-14T00:00:00   
3      Lights orbiting the moon I am 10,and i was wit...  2010-02-14T00:00:00   
4      3 Red lights in line pattern above El Cajon/ E...  2010-02-14T00:00:00   
...                                                  ...                  ...   
11656  Dozens of stars traveling east to North West, ...            2/07/2020   
11657  Looking out bedroom window towards the west. W...            2/07/2020   
11658  Blue Orb Sighting &quot;video recording&quot; ...              1/19/21   
11659  Red orb floats slowly changing direction then ...              1/31/20   
11660  It's too bad I can't tell the full story on he...            2/07/2020   

      city_latitude city_longitude       date     time  
0       32.80505053   -116.9175684  1/01/2010  0:00:00  
1       34.57594588   -92.63512706  1/01/2010  0:00:00  
2       32.80505053   -116.9175684  1/01/2010  0:00:00  
3           37.1336       -86.0986  1/01/2010  0:00:00  
4       32.7315897

In [6]:
ufo_data2020.to_csv("data/UFO_data_2010_2020v2.csv")